<a href="https://colab.research.google.com/github/qozm515/codex-playground/blob/main/MyPortfolioDashboard_v.5.4.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필수 라이브러리 설치
!pip -q install streamlit pandas numpy plotly openpyxl xlsxwriter lxml

In [ ]:
# 2. 성과 지표 모듈 작성 (metrics.py)
%%writefile metrics.py
import numpy as np
import pandas as pd

# -------- 유틸 --------
def _per_period_from_annual(r_annual, periods_per_year: int):
    """연(%) 또는 연수익률 시리즈/스칼라를 기간별 수익률로 변환"""
    if isinstance(r_annual, pd.Series):
        return (1.0 + r_annual.astype(float)) ** (1.0 / periods_per_year) - 1.0
    r = float(r_annual or 0.0)
    return (1.0 + r) ** (1.0 / periods_per_year) - 1.0

def _coerce_returns_from_value_series(series: pd.Series) -> pd.Series:
    s = pd.Series(series).dropna().astype(float)
    return s.pct_change().dropna()

def omega_ratio(r: pd.Series, threshold: float = 0.0) -> float:
    """Omega 비율: (r>th의 초과이득 합) / (r<th의 초과손실 합)"""
    if r is None or r.empty: return np.nan
    pos = (r[r > threshold] - threshold).sum()
    neg = (threshold - r[r < threshold]).sum()
    return np.nan if neg == 0 else float(pos / neg)

def drawdown_series_from_values(values: pd.Series) -> pd.Series:
    """가치시리즈 → 언더워터(피크대비 -DD) 시리즈"""
    if values is None or len(values) == 0:
        return pd.Series(dtype=float)
    v = pd.Series(values).dropna()
    if v.empty:
        return pd.Series(dtype=float)
    peak = v.cummax()
    dd = v / peak - 1.0
    return dd.fillna(0.0)

# -------- 기본 지표(하위호환) --------
def calculate_metrics(series: pd.Series, initial_capital: float, periods_per_year: int, rf_input=0.0):
    """
    (FinalBalance, TotalReturn, CAGR, Vol, Sharpe, MDD)
    rf_input: 0~1 스칼라(연) 또는 연(%) 시계열(pd.Series: 0~1)
    """
    s = pd.Series(series).dropna().astype(float)
    if s.size < 2 or initial_capital <= 0 or periods_per_year <= 0:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    fb = float(s.iloc[-1]); fr = fb / float(s.iloc[0]) - 1.0
    idx = pd.to_datetime(s.index)
    years = (idx[-1] - idx[0]).days / 365.2425
    cagr = (fb / float(s.iloc[0])) ** (1.0 / years) - 1.0 if years > 0 and s.iloc[0] > 0 else np.nan

    r = _coerce_returns_from_value_series(s)
    vol = r.std(ddof=0) * (periods_per_year ** 0.5) if not r.empty else np.nan

    # 초과수익(Sharpe용)
    if isinstance(rf_input, pd.Series):
        rf_series = rf_input.copy()
        if rf_series.abs().max() > 1.0: rf_series = rf_series / 100.0
        rf_pp = _per_period_from_annual(rf_series, periods_per_year).reindex(r.index).ffill().bfill()
        ex_r = r - rf_pp
    else:
        rf_annual = float(rf_input or 0.0)
        if rf_annual > 1.0: rf_annual = rf_annual / 100.0
        ex_r = r - _per_period_from_annual(rf_annual, periods_per_year)

    std_ex = ex_r.std(ddof=0)
    sharpe = (ex_r.mean() * periods_per_year) / (std_ex * (periods_per_year ** 0.5)) if std_ex > 0 else np.nan
    mdd = drawdown_series_from_values(s).min() if not s.empty else np.nan
    return fb, fr, cagr, vol, sharpe, mdd

# -------- 확장 지표(v4.x) --------
def calculate_metrics_v4(series: pd.Series,
                         initial_capital: float,
                         periods_per_year: int,
                         rf_input=0.0,
                         benchmark_series: pd.Series | None = None) -> dict:
    """
    확장된 지표 딕셔너리 반환
    Keys: Final, TotalReturn, CAGR, Vol, Sharpe, MDD, Calmar, Sortino, Omega,
          VaR_95, CVaR_95, Skew, Kurtosis, InfoRatio, Beta
    """
    out = {k: np.nan for k in [
        "Final","TotalReturn","CAGR","Vol","Sharpe","MDD","Calmar","Sortino","Omega",
        "VaR_95","CVaR_95","Skew","Kurtosis","InfoRatio","Beta"
    ]}
    s = pd.Series(series).dropna().astype(float)
    if s.size < 2 or initial_capital <= 0 or periods_per_year <= 0:
        return out

    fb, fr, cagr, vol, sharpe, mdd = calculate_metrics(s, initial_capital, periods_per_year, rf_input)
    out.update(dict(Final=fb, TotalReturn=fr, CAGR=cagr, Vol=vol, Sharpe=sharpe, MDD=mdd))
    out["Calmar"] = (cagr / abs(mdd)) if (pd.notna(cagr) and pd.notna(mdd) and mdd != 0) else np.nan

    r = _coerce_returns_from_value_series(s)
    if r.empty:
        return out

    # Rf per-period
    if isinstance(rf_input, pd.Series):
        rf_series = rf_input.copy()
        if rf_series.abs().max() > 1.0: rf_series = rf_series / 100.0
        rf_pp = _per_period_from_annual(rf_series, periods_per_year).reindex(r.index).ffill().bfill()
    else:
        rf_annual = float(rf_input or 0.0)
        if rf_annual > 1.0: rf_annual = rf_annual / 100.0
        rf_pp = pd.Series(_per_period_from_annual(rf_annual, periods_per_year), index=r.index)

    # Sortino (임계=Rf per period)
    ex_r_pp = r - rf_pp
    downside = ex_r_pp[ex_r_pp < 0]
    dstd = downside.std(ddof=0) * (periods_per_year ** 0.5) if not downside.empty else np.nan
    mu_ex_annual = ex_r_pp.mean() * periods_per_year
    out["Sortino"] = (mu_ex_annual / dstd) if (pd.notna(dstd) and dstd > 0) else np.nan

    # Omega (임계=0)
    out["Omega"] = omega_ratio(r, threshold=0.0)

    # 분포형 위험
    q05 = r.quantile(0.05)
    out["VaR_95"] = -q05 if pd.notna(q05) else np.nan
    tail = r[r <= q05]
    out["CVaR_95"] = -tail.mean() if not tail.empty else np.nan
    out["Skew"] = r.skew() if r.size > 2 else np.nan
    out["Kurtosis"] = r.kurtosis() if r.size > 3 else np.nan

    # 벤치마크 기반
    if benchmark_series is not None:
        rb = _coerce_returns_from_value_series(benchmark_series).reindex(r.index).dropna()
        r2 = r.reindex(rb.index).dropna()
        rb2 = rb.reindex(r2.index)
        if len(r2) > 2:
            active = r2 - rb2
            te = active.std(ddof=0) * (periods_per_year ** 0.5)
            mu_active_annual = active.mean() * periods_per_year
            out["InfoRatio"] = (mu_active_annual / te) if te and te > 0 else np.nan
            cov = r2.cov(rb2)
            var = rb2.var(ddof=0)
            out["Beta"] = (cov / var) if var and var > 0 else np.nan
    return out


Overwriting metrics.py


In [ ]:
# 3. Streamlit 앱 코드 작성 (app.py)
%%writefile app.py
# MyPortfolioDashboard - v5.4.5
# - Base: v5.4.4

import streamlit as st
import pandas as pd
import numpy as np
import io, json, math
from math import sqrt
from typing import Dict, Tuple, List, Optional
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import norm

# ---------- 공통 설정 ----------
VERSION = "5.4.5"
st.set_page_config(page_title=f"MyPortfolioDashboard v{VERSION}", layout="wide")

RET_CMAP = "RdYlGn"
CORR_CMAP = "RdBu_r"

# ---------- Session State ----------
def _init_state():
    S = st.session_state
    S.setdefault("portfolios", {})
    S.setdefault("initial_capital", 10_000_000)
    S.setdefault("rf_mode", "없음 (0%)")
    S.setdefault("rf_fixed_value", 2.00)
    S.setdefault("rf_series_data", None)
    S.setdefault("data_frequency", "일별")
    S.setdefault("rebalance_freq", "없음")
    S.setdefault("base_currency", "KRW")
    S.setdefault("price_data_info", {"loaded": False, "min_date": None, "max_date": None,
                                     "intersection_start": None, "intersection_end": None,
                                     "tickers": set(), "dataframe_raw": None})
    S.setdefault("exchange_rate_info", {"loaded": False, "series": None})
    S.setdefault("benchmark_ticker", "없음")
    S.setdefault("backtest_start_date", None)
    S.setdefault("backtest_end_date", None)
    S.setdefault("run_results", None)
    S.setdefault("__sig_price", None)
    S.setdefault("__sig_fx", None)
    S.setdefault("__sig_rf", None)
    S.setdefault("__export_sig", None)
    S.setdefault("__export_bytes", None)
    S.setdefault("chart_engine", "WebGL")
    S.setdefault("chart_quality", "고속(3k)")
    S.setdefault("ret_cmap_mode", "자동")
    S.setdefault("ret_cmap_min", None)
    S.setdefault("ret_cmap_max", None)
    S.setdefault("corr_cmap_mode", "자동")
    S.setdefault("corr_cmap_min", -1.0)
    S.setdefault("corr_cmap_max", 1.0)
    S.setdefault("growth_log", False)
    S.setdefault("growth_rfadj", False)
    S.setdefault("mr_page", 1)
    S.setdefault("mr_page_size", 12)
_init_state()

# ---------- 유틸 ----------
def _clean_date_col(s: pd.Series) -> pd.Series:
    return pd.to_datetime(
        s.astype(str).str.replace('년|월', '-', regex=True).str.replace('일', '', regex=False).str.strip(),
        errors="coerce"
    )
def _clean_num_col(s: pd.Series, allow_negative=False) -> pd.Series:
    if allow_negative:
        return pd.to_numeric(s.astype(str).str.replace(',', '', regex=False)
                             .str.replace(r"[^\d\.\-]", "", regex=True), errors="coerce")
    return pd.to_numeric(s.astype(str).str.replace(',', '', regex=False)
                         .str.replace(r"[^\d\.]", "", regex=True), errors="coerce")
def parse_weight_input(x) -> float:
    if x == "" or x is None: return 0.0
    try:
        v = float(str(x).replace(",", "").replace("%", "").strip())
        return max(0.0, min(100.0, v))
    except Exception:
        return 0.0
def _files_signature(files) -> Optional[str]:
    if not files: return None
    sig = []
    for f in files:
        size = getattr(f, "size", None)
        if size is None:
            try: size = len(f.getvalue())
            except Exception: size = 0
        sig.append(f"{f.name}:{int(size)}")
    return "|".join(sig)
def _file_signature(f) -> Optional[str]:
    if f is None: return None
    size = getattr(f, "size", None)
    if size is None:
        try: size = len(f.getvalue())
        except Exception: size = 0
    return f"{f.name}:{int(size)}"
def _toast_and_rerun(msg: str, icon="✅"):
    try: st.toast(msg, icon=icon)
    except Exception: st.success(msg)
    st.rerun()
def _month_diff(a: Optional[pd.Timestamp], b: Optional[pd.Timestamp]):
    if a is None or b is None: return None
    return (b.year - a.year) * 12 + (b.month - a.month)

# 색 범위 유틸(히트맵용)
def _auto_symmetric_range_from_values(values, default_half=0.01):
    try:
        arr = np.asarray(values, dtype=float)
        amax = float(np.nanmax(np.abs(arr)))
        if not np.isfinite(amax) or amax <= 0.0:
            amax = float(default_half)
    except Exception:
        amax = float(default_half)
    return [-amax, amax]
def _get_return_color_range_auto(values):
    return _auto_symmetric_range_from_values(values, default_half=0.01)
def _get_return_color_range_manual_or_auto(values):
    if st.session_state.ret_cmap_mode == "수동":
        vmin_pct = st.session_state.ret_cmap_min
        vmax_pct = st.session_state.ret_cmap_max
        if vmin_pct is not None and vmax_pct is not None and float(vmin_pct) < float(vmax_pct):
            return [float(vmin_pct)/100.0, float(vmax_pct)/100.0]
        else:
            st.warning("수익률 색 범위(%)가 올바르지 않아 자동 범위를 사용합니다.")
    return _get_return_color_range_auto(values)
def _get_corr_color_range():
    if st.session_state.corr_cmap_mode == "수동":
        vmin = st.session_state.corr_cmap_min
        vmax = st.session_state.corr_cmap_max
        try:
            vmin = float(vmin); vmax = float(vmax)
            if vmin < vmax:
                return [vmin, vmax]
        except Exception:
            pass
        st.warning("상관 색 범위가 올바르지 않아 자동 범위를 사용합니다.")
    return [-1.0, 1.0]

# ---------- 파일 로딩 ----------
@st.cache_data(show_spinner=False)
def _read_csv_two_col(file_bytes: bytes, enc="utf-8", header=0, names=None):
    bio = io.BytesIO(file_bytes)
    return pd.read_csv(bio, encoding=enc, header=header, names=names, usecols=[0,1], dtype=str)
@st.cache_data(show_spinner=False)
def _read_xlsx_two_col(file_bytes: bytes, header=0, names=None):
    bio = io.BytesIO(file_bytes)
    return pd.read_excel(bio, header=header, names=names, usecols=[0,1], dtype=str)
def _parse_two_col_file(file_name: str, file_bytes: bytes, value_col_name: str, allow_negative: bool) -> pd.Series:
    low = file_name.lower()
    if low.endswith((".xls", ".xlsx")):
        try:
            df = _read_xlsx_two_col(file_bytes, header=0)
            s = pd.Series(_clean_num_col(df.iloc[:,1], allow_negative=allow_negative).values,
                          index=_clean_date_col(df.iloc[:,0]), name=str(df.columns[1]))
        except Exception:
            df2 = _read_xlsx_two_col(file_bytes, header=None, names=["Date","Value"])
            s = pd.Series(_clean_num_col(df2["Value"], allow_negative=allow_negative).values,
                          index=_clean_date_col(df2["Date"]), name="Value")
        return s.dropna().sort_index().drop_duplicates()
    for enc in ("utf-8", "cp949"):
        try:
            df = _read_csv_two_col(file_bytes, enc=enc, header=0)
            s = pd.Series(_clean_num_col(df.iloc[:,1], allow_negative=allow_negative).values,
                          index=_clean_date_col(df.iloc[:,0]), name=str(df.columns[1]))
        except UnicodeDecodeError:
            continue
        except Exception:
            df2 = _read_csv_two_col(file_bytes, enc=enc, header=None, names=["Date","Value"])
            s = pd.Series(_clean_num_col(df2["Value"], allow_negative=allow_negative).values,
                          index=_clean_date_col(df2["Date"]), name="Value")
        return s.dropna().sort_index().drop_duplicates()
    raise ValueError(f"'{file_name}'에서 A=날짜, B=값 패턴을 찾지 못했습니다.")

@st.cache_data(show_spinner=False)
def combine_price_files_cached(file_blobs: tuple) -> tuple[pd.DataFrame, dict]:
    if not file_blobs:
        return pd.DataFrame(), {"loaded": False, "min_date": None, "max_date": None,
                                "intersection_start": None, "intersection_end": None, "tickers": set()}
    parsed = {}
    for name, blob in file_blobs:
        try:
            s = _parse_two_col_file(name, blob, "Price", allow_negative=False)
            if not s.empty: parsed[s.name] = s
        except Exception as e:
            st.warning(f"가격 파일 파싱 오류({name}): {e}")
    if not parsed:
        return pd.DataFrame(), {"loaded": False, "min_date": None, "max_date": None,
                                "intersection_start": None, "intersection_end": None, "tickers": set()}
    df = pd.concat(parsed.values(), axis=1)
    meta = {"loaded": True, "min_date": df.index.min().date(), "max_date": df.index.max().date(),
            "tickers": set(df.columns)}
    valid = [s.dropna() for s in parsed.values() if not s.empty]
    if valid:
        inter_start = max(s.first_valid_index() for s in valid)
        inter_end   = min(s.last_valid_index() for s in valid)
        if inter_start is not None and inter_end is not None and inter_start <= inter_end:
            meta["intersection_start"] = inter_start.date()
            meta["intersection_end"]   = inter_end.date()
    return df, meta

@st.cache_data(show_spinner=False)
def read_fx_series_cached(file_blob: tuple[str, bytes]) -> pd.Series:
    name, blob = file_blob
    s = _parse_two_col_file(name, blob, "Rate", allow_negative=False)
    s.name = "KRWUSD"
    return s
@st.cache_data(show_spinner=False)
def read_rf_series_cached(file_blob: tuple[str, bytes]) -> pd.Series:
    name, blob = file_blob
    return _parse_two_col_file(name, blob, "Rate", allow_negative=True)

# ---------- 전처리 ----------
def _rebalance_points(idx: pd.DatetimeIndex, freq_label: str) -> pd.DatetimeIndex:
    if freq_label == "매일": return idx
    if freq_label == "매월": return idx.to_series().resample("M").last().dropna().index.intersection(idx)
    if freq_label == "매분기": return idx.to_series().resample("Q").last().dropna().index.intersection(idx)
    if freq_label == "반기":
        m_ends = idx.to_series().resample("M").last().dropna().index
        keep = m_ends[m_ends.month.isin([6, 12])]
        return keep.intersection(idx)
    if freq_label == "매년": return idx.to_series().resample("Y").last().dropna().index.intersection(idx)
    return pd.DatetimeIndex([idx[0]])
@st.cache_data(show_spinner=False)
def preprocess_for_simulation(df_raw_all: pd.DataFrame, required_pairs: tuple, data_freq: str,
                              start_dt, end_dt, er_series: pd.Series | None, base_curr: str) -> pd.DataFrame:
    if not required_pairs: return pd.DataFrame()
    tickers = sorted({t for (t, fx) in required_pairs})
    df_raw = df_raw_all[[c for c in tickers if c in df_raw_all.columns]].copy()
    if df_raw.empty: return pd.DataFrame()
    df_raw = df_raw.apply(pd.to_numeric, errors="coerce")
    rule = {"일별":"D","주별":"W","월별":"M","연별":"Y"}.get(data_freq, "D")
    df_rs = df_raw if data_freq=="일별" else df_raw.resample(rule).last()
    last_valid = {t: (df_rs[t].dropna().index.max() if t in df_rs and df_rs[t].dropna().size else None)
                  for t in tickers}
    df_fill = df_rs.ffill()
    er = None; er_last = None
    if er_series is not None and not er_series.empty:
        er_raw = pd.to_numeric(er_series, errors="coerce")
        er_rs  = er_raw if data_freq=="일별" else er_raw.resample(rule).last()
        er_rs  = er_rs.reindex(df_fill.index)
        er_last = er_rs.dropna().index.max()
        er = er_rs.mask(~np.isfinite(er_rs) | (er_rs<=0)).ffill().bfill()
    df_fill = df_fill.loc[pd.to_datetime(start_dt):pd.to_datetime(end_dt)]
    if df_fill.empty: return pd.DataFrame()
    end_ts = pd.to_datetime(end_dt)
    pieces=[]
    def _cap_end(ticker: str, needs_er: bool):
        t_last = last_valid.get(ticker)
        if t_last is None: return None
        cap = min(t_last, end_ts)
        if needs_er and (er_last is not None): cap = min(cap, er_last)
        return cap
    if base_curr=="KRW":
        for (t, fx) in required_pairs:
            if t not in df_fill.columns: continue
            s = df_fill[t].copy()
            if fx and er is not None: s = s * er
            cap = _cap_end(t, needs_er=fx)
            if cap is not None: s.loc[s.index>cap] = np.nan
            s = s.mask(~np.isfinite(s) | (s<=0)); s.name=(t,bool(fx)); pieces.append(s)
    else:
        for (t, fx) in required_pairs:
            if t not in df_fill.columns: continue
            s = df_fill[t].copy()
            if (not fx) and (er is not None): s = s / er
            cap = _cap_end(t, needs_er=(not fx))
            if cap is not None: s.loc[s.index>cap] = np.nan
            s = s.mask(~np.isfinite(s) | (s<=0)); s.name=(t,bool(fx)); pieces.append(s)
    if not pieces: return pd.DataFrame()
    df_proc = pd.concat(pieces, axis=1)
    df_proc.columns = pd.MultiIndex.from_tuples(df_proc.columns, names=["Ticker","FX"])
    return df_proc.dropna(axis=1, how="all").dropna(how="all")

# ---------- 시뮬레이션 ----------
def run_backtest_simulation(df_proc: pd.DataFrame, weights_struct: dict, pf_name: str,
                            rebalance_freq: str, initial_capital: float) -> Optional[pd.Series]:
    if df_proc is None or df_proc.empty: return None
    want={}
    for t,d in weights_struct.items():
        w=float(d.get("w",0.0)); fx=bool(d.get("fx",False))
        if w>0: want[(t,fx)] = want.get((t,fx),0.0) + w
    cols=[c for c in df_proc.columns if (c[0],c[1]) in want and want[(c[0],c[1])]>0]
    if not cols: return None
    sub=df_proc.loc[:,cols].copy().ffill().dropna(how="any")
    if sub.empty: return None
    W=pd.Series({c:want[(c[0],c[1])]/100.0 for c in cols}, index=sub.columns)
    if not np.isclose(W.sum(),1.0): W=W/W.sum()
    X=sub.to_numpy(dtype=float); idx=sub.index
    if rebalance_freq=="매일":
        if len(X)<2: return None
        R=X[1:]/X[:-1]-1.0
        port_R=R@W.to_numpy()
        curve=float(initial_capital)*np.cumprod(1.0+port_R)
        ser=pd.Series(curve, index=idx[1:], name=pf_name)
        return pd.concat([pd.Series([float(initial_capital)], index=[idx[0]-pd.Timedelta(days=1)]), ser])
    if rebalance_freq=="없음":
        first=X[0]
        if (first<=0).any() or (not np.isfinite(first).all()): return None
        units=(float(initial_capital)*W.to_numpy())/first
        values=(X*units).sum(axis=1)
        ser=pd.Series(values, index=idx, name=pf_name)
        return pd.concat([pd.Series([float(initial_capital)], index=[idx[0]-pd.Timedelta(days=1)]), ser])
    rpoints=_rebalance_points(idx, rebalance_freq)
    cut=pd.Index(sorted(set([idx[0]])|set(rpoints)|set([idx[-1]]))).intersection(idx)
    cap=float(initial_capital); segs=[]
    for i in range(len(cut)-1):
        a,b=cut[i],cut[i+1]
        seg=sub.loc[a:b]
        if seg.empty: continue
        Xs=seg.to_numpy(dtype=float); start=Xs[0]
        if (start<=0).any() or (not np.isfinite(start).all()): continue
        units=(cap*W.to_numpy())/start
        val=(Xs*units).sum(axis=1)
        s=pd.Series(val, index=seg.index)
        s = s if i>0 else pd.concat([pd.Series([cap], index=[idx[0]-pd.Timedelta(days=1)]), s])
        if i>0: s=s.iloc[1:]
        segs.append(s); cap=float(s.iloc[-1])
    if not segs: return None
    ser=pd.concat(segs); ser.name=pf_name
    return ser

# ---------- 메트릭(요약) ----------
def calculate_metrics_v4(values: pd.Series, init_cap: float, periods_per_year: int,
                         rf_input, bench_values: Optional[pd.Series]):
    v=values.astype(float).dropna()
    total_ret=float(v.iloc[-1]/v.iloc[0]-1.0) if v.size>1 else np.nan
    R=v.pct_change().dropna()
    if isinstance(rf_input,(int,float,np.floating)): rf=rf_input/periods_per_year
    elif isinstance(rf_input,pd.Series):
        rf=rf_input.copy(); rf.index=pd.to_datetime(rf.index)
        freq={1:"Y",12:"M",52:"W",252:"D"}[periods_per_year]
        rf=rf.resample(freq).last()/100.0
        rf=rf.reindex(R.index).ffill().bfill().fillna(0.0)
    else: rf=0.0
    ex=R-(rf if isinstance(rf,pd.Series) else rf)
    mu=float(R.mean()); vol=float(R.std(ddof=1))
    cagr=(1.0+mu)**periods_per_year-1.0 if np.isfinite(mu) else np.nan
    sharpe=float(ex.mean()/ex.std(ddof=1))*sqrt(periods_per_year) if ex.std(ddof=1)>0 else np.nan
    downside=np.minimum(0.0, R-0.0)
    sortino=float((R.mean())/downside.std(ddof=1))*sqrt(periods_per_year) if downside.std(ddof=1)>0 else np.nan
    runmax=v.cummax(); mdd=float((v/runmax-1.0).min())
    calmar=(cagr/abs(mdd)) if (np.isfinite(cagr) and mdd<0) else np.nan
    omega=np.nan
    var_95=-(np.nanpercentile(R,5))
    cvar_95=float(-R[R<=np.nanpercentile(R,5)].mean()) if (R<=np.nanpercentile(R,5)).any() else np.nan
    skew=float(R.skew()); kurt=float(R.kurt())
    info_ratio=beta=np.nan
    if bench_values is not None:
        b=bench_values.reindex(v.index).pct_change().dropna()
        df=pd.concat([R.rename("p"), b.rename("b")], axis=1).dropna()
        if not df.empty:
            diff=df["p"]-df["b"]
            te=float(diff.std(ddof=1))*sqrt(periods_per_year)
            act=float(diff.mean())*periods_per_year
            info_ratio=act/te if te and np.isfinite(te) and te!=0 else np.nan
            varb=float(df["b"].var(ddof=1))
            beta=float(df["p"].cov(df["b"])/varb) if varb>0 else np.nan
    return {"Final":float(v.iloc[-1]), "TotalReturn":total_ret, "CAGR":cagr, "Vol":vol*sqrt(periods_per_year),
            "Sharpe":sharpe, "Sortino":sortino, "MDD":mdd, "Calmar":calmar, "Omega":omega,
            "VaR_95":var_95, "CVaR_95":cvar_95, "Skew":skew, "Kurtosis":kurt,
            "InfoRatio":info_ratio, "Beta":beta}

# ---------- 사이드바 ----------
with st.sidebar:
    st.header(f"데이터 업로드 · v{VERSION}")
    price_files = st.file_uploader("가격 파일(여러 개 가능, A=날짜 B=가격)",
                                   type=["csv","xlsx","xls"], accept_multiple_files=True, key="price_files")
    fx_file = st.file_uploader("KRW/USD 환율 파일 (A=날짜 B=환율)", type=["csv","xlsx","xls"], key="fx_file")
    if price_files:
        new_sig=_files_signature(price_files)
        if st.session_state.__sig_price!=new_sig:
            blobs=tuple((f.name,f.getvalue()) for f in price_files)
            all_df, meta = combine_price_files_cached(blobs)
            st.session_state.price_data_info.update({**meta, "dataframe_raw": all_df})
            st.session_state.__sig_price=new_sig
            st.session_state.run_results=None; st.session_state.__export_sig=None; st.session_state.__export_bytes=None
    if fx_file is not None:
        sig=_file_signature(fx_file)
        if st.session_state.__sig_fx!=sig:
            s=read_fx_series_cached((fx_file.name, fx_file.getvalue()))
            st.session_state.exchange_rate_info={"loaded":True,"series":s}; st.session_state.__sig_fx=sig
            st.session_state.run_results=None; st.session_state.__export_sig=None; st.session_state.__export_bytes=None
    else:
        st.session_state.exchange_rate_info={"loaded":False,"series":None}; st.session_state.__sig_fx=None
    st.header("포트폴리오 저장/불러오기")
    if st.session_state.portfolios:
        st.download_button("현재 포트폴리오(.json) 저장",
                           data=json.dumps(st.session_state.portfolios, indent=2, ensure_ascii=False),
                           file_name="my_portfolios.json", mime="application/json", use_container_width=True)
    pf_json = st.file_uploader("포트폴리오(.json) 불러오기", type=["json"], key="pf_json_sidebar")
    if st.button("불러오기", use_container_width=True):
        if pf_json is None: st.warning("파일을 선택하십시오.")
        else:
            data=json.loads(pf_json.getvalue().decode("utf-8"))
            new={}
            for name,payload in data.items():
                if payload and isinstance(next(iter(payload.values())), dict) and "w" in next(iter(payload.values())):
                    new[name]=payload
                else:
                    new[name]={t:{"w":float(w),"fx":False} for t,w in payload.items()}
            st.session_state.portfolios.update(new); _toast_and_rerun(f"{len(new)}개 포트폴리오 로드 완료")

st.title(f"MyPortfolioDashboard v{VERSION}")

# ---------- 데이터 준비 ----------
st.header("데이터 준비")
pinfo=st.session_state.price_data_info
if pinfo["loaded"] and pinfo["intersection_start"] and pinfo["intersection_end"]:
    st.session_state.backtest_start_date = st.session_state.backtest_start_date or pinfo["intersection_start"]
    st.session_state.backtest_end_date   = st.session_state.backtest_end_date   or pinfo["intersection_end"]
    c1,c2=st.columns(2)
    with c1: st.date_input("시작일", key="backtest_start_date",
                           min_value=pinfo["intersection_start"], max_value=pinfo["intersection_end"])
    with c2: st.date_input("종료일", key="backtest_end_date",
                           min_value=pinfo["intersection_start"], max_value=pinfo["intersection_end"])
else:
    st.info("사이드바에서 가격/환율 파일을 업로드하십시오.")

# ---------- 포트폴리오 관리 ----------
st.header("포트폴리오 관리")
left,right=st.columns(2, gap="large")
with left:
    st.subheader("새 포트폴리오 만들기")
    known=sorted(list(pinfo["tickers"]))
    init_df = (pd.DataFrame({"Ticker": known, "Weight(%)": [0.0]*len(known), "FX":[False]*len(known)})
               if known else pd.DataFrame(columns=["Ticker","Weight(%)","FX"]))
    with st.form("new_pf_form_table", clear_on_submit=True):
        pf_name = st.text_input("포트폴리오 이름", placeholder="예: 60/40 기본")
        st.caption("가중치(%)와 FX(환노출)를 표에서 입력하십시오. 0%는 저장 시 제외됩니다.")
        edited = st.data_editor(init_df, use_container_width=True, hide_index=True,
                                column_config={
                                    "Ticker": st.column_config.TextColumn(disabled=True),
                                    "Weight(%)": st.column_config.NumberColumn(min_value=0.0,max_value=100.0,step=1.0,format="%.2f"),
                                    "FX": st.column_config.CheckboxColumn()
                                }, key="new_pf_editor")
        if st.form_submit_button("저장"):
            try:
                name = (pf_name or "").strip()
                if not name: raise ValueError("이름을 입력하십시오.")
                weights={}
                for _,row in edited.iterrows():
                    w=parse_weight_input(row.get("Weight(%)",0))
                    if w>0:
                        t=str(row.get("Ticker")); fx=bool(row.get("FX",False))
                        weights[t]={"w":w,"fx":fx}
                if not weights: raise ValueError("가중치가 모두 0%입니다.")
                st.session_state.portfolios[name]=weights; _toast_and_rerun(f"'{name}' 저장 완료")
            except Exception as e:
                st.error(f"입력 오류: {e}")
with right:
    st.subheader("저장된 포트폴리오")
    if not st.session_state.portfolios: st.info("없음.")
    else:
        for pf_name,pf_struct in list(st.session_state.portfolios.items()):
            with st.expander(f"{pf_name}", expanded=False):
                df_view=pd.DataFrame([{"Ticker":t,"Weight(%)":pf_struct[t].get("w",0.0),"FX":pf_struct[t].get("fx",False)}
                                      for t in list(pf_struct.keys())])
                edited=st.data_editor(df_view, use_container_width=True, hide_index=True,
                                      column_config={
                                          "Ticker": st.column_config.TextColumn(disabled=True),
                                          "Weight(%)": st.column_config.NumberColumn(min_value=0.0,max_value=100.0,step=1.0,format="%.2f"),
                                          "FX": st.column_config.CheckboxColumn()
                                      }, key=f"edit_{pf_name}")
                c1,c2,c3=st.columns([1,1,1])
                with c1:
                    new_name=st.text_input("이름 변경", value=pf_name, key=f"rename_{pf_name}")
                with c2:
                    if st.button("수정/저장", key=f"save_{pf_name}", use_container_width=True):
                        try:
                            updated={}
                            for _,row in edited.iterrows():
                                w=parse_weight_input(row.get("Weight(%)",0))
                                if w>0:
                                    t=str(row.get("Ticker")); fx=bool(row.get("FX",False))
                                    updated[t]={"w":w,"fx":fx}
                            if not updated: raise ValueError("가중치가 모두 0%입니다.")
                            if new_name!=pf_name and pf_name in st.session_state.portfolios:
                                del st.session_state.portfolios[pf_name]
                            st.session_state.portfolios[new_name]=updated; _toast_and_rerun(f"'{new_name}' 저장/수정 완료")
                        except Exception as e:
                            st.error(f"저장 오류: {e}")
                with c3:
                    if st.button("삭제", key=f"del_{pf_name}", type="secondary", use_container_width=True):
                        if pf_name in st.session_state.portfolios:
                            del st.session_state.portfolios[pf_name]; _toast_and_rerun(f"'{pf_name}' 삭제 완료")

# ---------- 설정 ----------
st.header("설정")
colA,colB,colC=st.columns(3, gap="large")
with colA:
    st.subheader("기준 설정")
    st.radio("기준 통화", options=("KRW","USD"), key="base_currency", horizontal=True)
    st.selectbox("데이터 주기", options=("일별","주별","월별","연별"), key="data_frequency")
    st.selectbox("리밸런싱 주기", options=("없음","매일","매월","매분기","반기","매년"), key="rebalance_freq")
    curr_sym="₩" if st.session_state.base_currency=="KRW" else "$"
    st.number_input(f"초기 투자금 ({curr_sym})", min_value=100, step=1000, key="initial_capital", format="%d")
with colB:
    st.subheader("무위험 수익률(샤프/소르티노)")
    st.radio("방식", options=("없음 (0%)","고정값 (%)","CSV 업로드 (^IRX 등)"), key="rf_mode")
    st.session_state.rf_series_data=None
    if st.session_state.rf_mode=="고정값 (%)":
        st.number_input("연(%)", min_value=0.0, step=0.1, format="%.2f", key="rf_fixed_value")
    elif st.session_state.rf_mode=="CSV 업로드 (^IRX 등)":
        rf_file=st.file_uploader("Rf 파일 업로드 (A=날짜 B=연%)", type=["csv","xlsx","xls"], key="rf_file_main")
        sig=_file_signature(rf_file)
        if rf_file is not None and st.session_state.__sig_rf!=sig:
            try:
                rf_series=read_rf_series_cached((rf_file.name, rf_file.getvalue()))
                st.session_state.rf_series_data=rf_series; st.session_state.__sig_rf=sig; st.success("Rf 데이터 로드 완료")
            except Exception as e:
                st.warning(f"Rf 파일 오류: {e}"); st.session_state.rf_series_data=None
        elif rf_file is None: st.session_state.__sig_rf=None
with colC:
    st.subheader("벤치마크")
    known=sorted(list(pinfo["tickers"]))
    for pf in st.session_state.portfolios.values():
        for k in pf.keys():
            if k not in known: known.append(k)
    bench_options=["없음"]+known
    st.selectbox("벤치마크 티커", options=bench_options, key="benchmark_ticker")

# ---------- 백테스트 ----------
st.header("백테스트")
if st.button("백테스트 실행", use_container_width=True):
    try:
        if not pinfo["loaded"]: raise ValueError("가격 데이터가 없습니다.")
        if not st.session_state.portfolios: raise ValueError("포트폴리오를 먼저 저장하십시오.")
        start_dt,end_dt=st.session_state.backtest_start_date, st.session_state.backtest_end_date
        if start_dt is None or end_dt is None: raise ValueError("기간을 선택하십시오.")
        if start_dt>end_dt: raise ValueError("시작일이 종료일보다 큽니다.")
        base_curr=st.session_state.base_currency
        rebalance=st.session_state.rebalance_freq
        init_cap=float(st.session_state.initial_capital)
        data_freq=st.session_state.data_frequency
        df_raw_all=pinfo["dataframe_raw"]
        required_pairs=set(); used_cols=set()
        for pf in st.session_state.portfolios.values():
            for t,d in pf.items():
                if d.get("w",0)>0:
                    used_cols.add(t); required_pairs.add((t,bool(d.get("fx",False))))
        bm=st.session_state.benchmark_ticker
        if bm!="없음":
            used_cols.add(bm)
            bm_fx=False
            for pf in st.session_state.portfolios.values():
                if bm in pf: bm_fx=bool(pf[bm].get("fx",False)); break
            required_pairs.add((bm,bm_fx))
        if not used_cols: raise ValueError("사용할 티커가 없습니다.")
        er = st.session_state.exchange_rate_info["series"].copy() if st.session_state.exchange_rate_info["loaded"] else None
        need_er = any(fx for (_,fx) in required_pairs) if base_curr=="KRW" else any((not fx) for (_,fx) in required_pairs)
        if need_er and er is None: raise ValueError("환율 데이터가 필요합니다. KRW/USD 환율 파일을 업로드하십시오.")
        req_pairs_sorted=tuple(sorted(required_pairs, key=lambda x:(x[0],x[1])))
        df_proc=preprocess_for_simulation(df_raw_all, req_pairs_sorted, data_freq, start_dt, end_dt, er, base_curr)
        if df_proc.empty: raise ValueError("선택 기간 내 데이터가 없습니다.")
        gs,ge=df_proc.index.min(), df_proc.index.max()
        bench_values=None
        if bm!="없음":
            bench_struct={bm:{"w":100.0,"fx":next((fx for (t,fx) in req_pairs_sorted if t==bm), False)}}
            bench_values=run_backtest_simulation(df_proc, bench_struct, "Benchmark", "없음", init_cap)
        values_dict={}
        for name,wstruct in st.session_state.portfolios.items():
            series=run_backtest_simulation(df_proc, wstruct, name, rebalance, init_cap)
            if series is not None: values_dict[name]=series
        periods_per_year={"일별":252,"주별":52,"월별":12,"연별":1}[data_freq]
        if st.session_state.rf_mode=="고정값 (%)": rf_input=st.session_state.rf_fixed_value/100.0
        elif st.session_state.rf_mode=="CSV 업로드 (^IRX 등)" and st.session_state.rf_series_data is not None:
            rf_input=st.session_state.rf_series_data
        else: rf_input=0.0
        results_table=[]
        if bench_values is not None:
            m=calculate_metrics_v4(bench_values, init_cap, periods_per_year, rf_input, None)
            results_table.append({"포트폴리오":f"벤치마크 ({bm})","초기 금액":init_cap,"최종 금액":m["Final"],
                                  "총수익률":f"{m['TotalReturn']:.2%}","CAGR":f"{m['CAGR']:.2%}",
                                  "변동성":f"{m['Vol']:.2%}","샤프":f"{m['Sharpe']:.2f}" if pd.notna(m["Sharpe"]) else "N/A",
                                  "MDD":f"{m['MDD']:.2%}","Calmar":f"{m['Calmar']:.2f}" if pd.notna(m["Calmar"]) else "N/A",
                                  "Sortino":f"{m['Sortino']:.2f}" if pd.notna(m["Sortino"]) else "N/A",
                                  "Omega":"N/A","VaR(95)":f"{m['VaR_95']:.2%}","CVaR(95)":f"{m['CVaR_95']:.2%}",
                                  "Skew":f"{m['Skew']:.2f}","Kurtosis":f"{m['Kurtosis']:.2f}",
                                  "InfoRatio":"N/A","Beta":"N/A"})
        for name,s in values_dict.items():
            m=calculate_metrics_v4(s, init_cap, periods_per_year, rf_input, bench_values)
            results_table.append({"포트폴리오":name,"초기 금액":init_cap,"최종 금액":m["Final"],
                                  "총수익률":f"{m['TotalReturn']:.2%}","CAGR":f"{m['CAGR']:.2%}",
                                  "변동성":f"{m['Vol']:.2%}","샤프":f"{m['Sharpe']:.2f}" if pd.notna(m["Sharpe"]) else "N/A",
                                  "MDD":f"{m['MDD']:.2%}","Calmar":f"{m['Calmar']:.2f}" if pd.notna(m["Calmar"]) else "N/A",
                                  "Sortino":f"{m['Sortino']:.2f}" if pd.notna(m["Sortino"]) else "N/A",
                                  "Omega":"N/A","VaR(95)":f"{m['VaR_95']:.2%}","CVaR(95)":f"{m['CVaR_95']:.2%}",
                                  "Skew":f"{m['Skew']:.2f}","Kurtosis":f"{m['Kurtosis']:.2f}",
                                  "InfoRatio":f"{m['InfoRatio']:.2f}" if pd.notna(m["InfoRatio"]) else "N/A",
                                  "Beta":f"{m['Beta']:.2f}" if pd.notna(m["Beta"]) else "N/A"})
        prices_used=pinfo["dataframe_raw"].copy().loc[gs:ge]
        st.session_state.run_results={"metrics_df":(pd.DataFrame(results_table).set_index("포트폴리오")
                                       if results_table else pd.DataFrame()),
                                      "values_dict":values_dict,"bench_values":bench_values,
                                      "global_start":gs,"global_end":ge,"base_currency":st.session_state.base_currency,
                                      "prices_raw_used":prices_used,"data_frequency":data_freq,
                                      "rebalance_freq":st.session_state.rebalance_freq,"df_proc_used":df_proc,
                                      "req_pairs":req_pairs_sorted}
        st.success("백테스트 완료")
    except Exception as e:
        st.session_state.run_results=None; st.session_state.__export_sig=None; st.session_state.__export_bytes=None
        st.error(f"오류: {e}")

# ---------- 결과 ----------
res=st.session_state.get("run_results")
if not res:
    st.info("백테스트를 먼저 실행하십시오.")
else:
    gs,ge=res["global_start"],res["global_end"]
    base_ccy=st.session_state.base_currency
    sym="₩" if base_ccy=="KRW" else "$"
    values_dict=res["values_dict"]; bench_values=res["bench_values"]
    df_proc_used=res.get("df_proc_used")
    data_freq=res["data_frequency"]
    # 공통 보조: 자산 라벨링 및 월/연 수익률 생성
    def _asset_series_from_df(df: pd.DataFrame) -> dict[str, pd.Series]:
        assets = {}
        if df is None or df.empty:
            return assets
        for col in df.columns:
            label = None
            if isinstance(col, tuple) and len(col) == 2:
                t, fx = col
                label = f"{t}{'(H)' if (not fx and st.session_state.base_currency!='USD') else ''}"
            else:
                label = str(col)
            s = pd.to_numeric(df[col], errors="coerce").dropna()
            if not s.empty:
                assets[label] = s
        return assets

    def _monthly_returns_df(assets: dict[str, pd.Series]) -> pd.DataFrame:
        frames = []
        for name, s in assets.items():
            m = s.resample("M").last().pct_change()
            frames.append(m.rename(name))
        return pd.concat(frames, axis=1).dropna(how="all") if frames else pd.DataFrame()

    def _annual_returns_df(assets: dict[str, pd.Series]) -> pd.DataFrame:
        frames = []
        for name, s in assets.items():
            y = s.resample("Y").last().pct_change()
            frames.append(y.rename(name))
        return pd.concat(frames, axis=1).dropna(how="all") if frames else pd.DataFrame()

    # 자산 원천 시리즈/월수익/연수익 준비
    _assets = _asset_series_from_df(df_proc_used)
    _mret_assets = _monthly_returns_df(_assets)      # 섹션 11·12·13·14
    _yret_assets = _annual_returns_df(_assets)       # 섹션 15

    # 포트폴리오 월수익(히트맵/상관/분해용)
    _mret_ports = {}
    for pname, pval in values_dict.items():
        _mret_ports[pname] = pval.resample("M").last().pct_change().dropna()

    # 롤링/연환산 계산 유틸
    def _rolling_ann_from_monthly(m: pd.Series, years: int) -> pd.Series:
        win = int(years * 12)
        m = pd.to_numeric(m, errors="coerce").dropna()
        if m.size < win:
            return pd.Series(dtype=float, index=m.index)
        comp = (1.0 + m).rolling(window=win, min_periods=win).apply(
            lambda x: np.prod(x) - 1.0, raw=True
        )
        ann = (1.0 + comp) ** (12.0 / win) - 1.0
        return ann.dropna()

    def _rolling_table_for_portfolios(mret_ports: dict[str, pd.Series], years_list=(1,3,5,7)) -> pd.DataFrame:
        rows = []
        for y in years_list:
            row = {"Roll Period": f"{y} year" if y == 1 else f"{y} years"}
            for pname, m in mret_ports.items():
                ann = _rolling_ann_from_monthly(m, y)
                if ann.empty:
                    row[(pname, "Average")] = "N/A"
                    row[(pname, "High")] = "N/A"
                    row[(pname, "Low")] = "N/A"
                else:
                    row[(pname, "Average")] = f"{float(ann.mean()):.2%}"
                    row[(pname, "High")]    = f"{float(ann.max()):.2%}"
                    row[(pname, "Low")]     = f"{float(ann.min()):.2%}"
            rows.append(row)
        df = pd.DataFrame(rows)
        ordered = ["Roll Period"]
        for pname in mret_ports.keys():
            ordered += [(pname, "Average"), (pname, "High"), (pname, "Low")]
        keep = [c for c in ordered if c in df.columns]
        return df[keep] if keep else df

    def _rolling_ann_long_df(mret_ports: dict[str, pd.Series], years: int) -> pd.DataFrame:
        rec=[]
        for pname, m in mret_ports.items():
            ann = _rolling_ann_from_monthly(m, years)
            for dt, val in ann.items():
                rec.append({"Date": dt, "Portfolio": pname, "AnnReturn": float(val)})
        return pd.DataFrame(rec)

    # ===== Bench/Trailing/Active 보조 =====
    _mret_bench = None
    _yret_bench = None
    if bench_values is not None and not bench_values.empty:
        _mret_bench = bench_values.resample("M").last().pct_change().dropna()
        _yret_bench = bench_values.resample("Y").last().pct_change().dropna()

    _yret_ports = {n: s.resample("Y").last().pct_change().dropna()
                   for n, s in values_dict.items()}

    def _total_return_over_months(m: pd.Series, months: int) -> float | None:
        if m is None or m.empty or len(m) < months: return None
        sub = m.iloc[-months:]
        return float(np.prod(1.0 + sub) - 1.0)

    def _ytd_total_return(m: pd.Series) -> float | None:
        if m is None or m.empty: return None
        last = m.index[-1]
        y0 = pd.Timestamp(year=last.year, month=1, day=1)
        rng = m.loc[(m.index >= y0) & (m.index <= last)]
        if rng.empty: return None
        return float(np.prod(1.0 + rng) - 1.0)

    def _ann_return_from_monthly(m: pd.Series, years: int) -> float | None:
        need = years * 12
        if m is None or m.empty or len(m) < need: return None
        sub = m.iloc[-need:]
        comp = float(np.prod(1.0 + sub) - 1.0)
        return (1.0 + comp) ** (12.0 / (years * 12)) - 1.0

    from math import sqrt
    def _ann_std_from_monthly(m: pd.Series, years: int) -> float | None:
        need = years * 12
        if m is None or m.empty or len(m) < need: return None
        sub = m.iloc[-need:]
        sd_m = float(np.std(sub, ddof=1)) if len(sub) > 1 else np.nan
        return sd_m * sqrt(12.0)

    def _active_monthly(m_port: pd.Series, m_bench: pd.Series) -> pd.Series:
        if m_port is None or m_bench is None: return pd.Series(dtype=float)
        j = m_port.dropna().index.intersection(m_bench.dropna().index)
        return (m_port.reindex(j) - m_bench.reindex(j)).dropna()

    def _tracking_error(m_active: pd.Series, window: int) -> pd.Series:
        if m_active is None or m_active.empty: return pd.Series(dtype=float)
        sd = m_active.rolling(window=window, min_periods=window).std(ddof=1)
        return (sd * sqrt(12.0)).dropna()

    def _rolling_active_ann(m_active: pd.Series, window: int) -> pd.Series:
        if m_active is None or m_active.empty: return pd.Series(dtype=float)
        mu = m_active.rolling(window=window, min_periods=window).mean()
        return (mu * 12.0).dropna()

    def _up_down_table(m_port: pd.Series, m_bench: pd.Series) -> dict:
        if m_port is None or m_bench is None: return {}
        j = m_port.dropna().index.intersection(m_bench.dropna().index)
        rp = m_port.reindex(j); rb = m_bench.reindex(j)
        up = rb > 0; dn = rb <= 0
        def stat(mask):
            arr = (rp - rb)[mask]
            if arr.size == 0: return (0, 0, 0, np.nan, np.nan, np.nan)
            above = int((arr > 0).sum()); below = int((arr <= 0).sum()); tot = int(arr.size)
            avg_above = float(arr[arr > 0].mean()) if (arr > 0).any() else np.nan
            avg_below = float(arr[arr <= 0].mean()) if (arr <= 0).any() else np.nan
            return above, below, tot, avg_above, avg_below, float(arr.mean())
        au, bu, tu, u_above, u_below, u_avg = stat(up)
        ad, bd, td, d_above, d_below, d_avg = stat(dn)
        return {
            "up": {"above": au, "below": bu, "total": tu, "avg_above": u_above, "avg_below": u_below, "avg_total": u_avg},
            "down": {"above": ad, "below": bd, "total": td, "avg_above": d_above, "avg_below": d_below, "avg_total": d_avg},
            "total": {"above": au+ad, "below": bu+bd, "total": tu+td,
                      "avg_above": np.nanmean([u_above, d_above]), "avg_below": np.nanmean([u_below, d_below]),
                      "avg_total": np.nanmean([u_avg, d_avg])},
        }

    def _return_vs_benchmark_bins(m_port: pd.Series, m_bench: pd.Series, step=0.01) -> pd.DataFrame:
        if m_port is None or m_bench is None: return pd.DataFrame()
        j = m_port.dropna().index.intersection(m_bench.dropna().index)
        df = pd.DataFrame({"bench": m_bench.reindex(j), "port": m_port.reindex(j)}).dropna()
        bins = np.arange(np.floor(df["bench"].min()/step)*step, np.ceil(df["bench"].max()/step)*step + step, step)
        df["bin"] = pd.cut(df["bench"], bins=bins, include_lowest=True)
        g = df.groupby("bin", observed=True).agg({"bench":"mean", "port":"mean"}).dropna().reset_index()
        g["label"] = g["bin"].astype(str)
        return g

    # ===== (중복 정의 허용) Bench/Trailing/Active 보조 재정의 블록 — 유지 =====
    _mret_bench = None
    _yret_bench = None
    if bench_values is not None and not bench_values.empty:
        _mret_bench = bench_values.resample("M").last().pct_change().dropna()
        _yret_bench = bench_values.resample("Y").last().pct_change().dropna()

    _yret_ports = {n: s.resample("Y").last().pct_change().dropna()
                   for n, s in values_dict.items()}

    def _total_return_over_months(m: pd.Series, months: int) -> float | None:
        if m is None or m.empty or len(m) < months: return None
        sub = m.iloc[-months:]
        return float(np.prod(1.0 + sub) - 1.0)

    def _ytd_total_return(m: pd.Series) -> float | None:
        if m is None or m.empty: return None
        last = m.index[-1]
        y0 = pd.Timestamp(year=last.year, month=1, day=1)
        rng = m.loc[(m.index >= y0) & (m.index <= last)]
        if rng.empty: return None
        return float(np.prod(1.0 + rng) - 1.0)

    def _ann_return_from_monthly(m: pd.Series, years: int) -> float | None:
        need = years * 12
        if m is None or m.empty or len(m) < need: return None
        sub = m.iloc[-need:]
        comp = float(np.prod(1.0 + sub) - 1.0)
        return (1.0 + comp) ** (12.0 / (years * 12)) - 1.0

    from math import sqrt
    def _ann_std_from_monthly(m: pd.Series, years: int) -> float | None:
        need = years * 12
        if m is None or m.empty or len(m) < need: return None
        sub = m.iloc[-need:]
        sd_m = float(np.std(sub, ddof=1)) if len(sub) > 1 else np.nan
        return sd_m * sqrt(12.0)

    def _active_monthly(m_port: pd.Series, m_bench: pd.Series) -> pd.Series:
        if m_port is None or m_bench is None: return pd.Series(dtype=float)
        j = m_port.dropna().index.intersection(m_bench.dropna().index)
        return (m_port.reindex(j) - m_bench.reindex(j)).dropna()

    def _tracking_error(m_active: pd.Series, window: int) -> pd.Series:
        if m_active is None or m_active.empty: return pd.Series(dtype=float)
        sd = m_active.rolling(window=window, min_periods=window).std(ddof=1)
        return (sd * sqrt(12.0)).dropna()

    def _rolling_active_ann(m_active: pd.Series, window: int) -> pd.Series:
        if m_active is None or m_active.empty: return pd.Series(dtype=float)
        mu = m_active.rolling(window=window, min_periods=window).mean()
        return (mu * 12.0).dropna()

    def _up_down_table(m_port: pd.Series, m_bench: pd.Series) -> dict:
        if m_port is None or m_bench is None: return {}
        j = m_port.dropna().index.intersection(m_bench.dropna().index)
        rp = m_port.reindex(j); rb = m_bench.reindex(j)
        up = rb > 0; dn = rb <= 0
        def stat(mask):
            arr = (rp - rb)[mask]
            if arr.size == 0: return (0, 0, 0, np.nan, np.nan, np.nan)
            above = int((arr > 0).sum()); below = int((arr <= 0).sum()); tot = int(arr.size)
            avg_above = float(arr[arr > 0].mean()) if (arr > 0).any() else np.nan
            avg_below = float(arr[arr <= 0].mean()) if (arr <= 0).any() else np.nan
            return above, below, tot, avg_above, avg_below, float(arr.mean())
        au, bu, tu, u_above, u_below, u_avg = stat(up)
        ad, bd, td, d_above, d_below, d_avg = stat(dn)
        return {
            "up": {"above": au, "below": bu, "total": tu, "avg_above": u_above, "avg_below": u_below, "avg_total": u_avg},
            "down": {"above": ad, "below": bd, "total": td, "avg_above": d_above, "avg_below": d_below, "avg_total": d_avg},
            "total": {"above": au+ad, "below": bu+bd, "total": tu+td,
                      "avg_above": np.nanmean([u_above, d_above]), "avg_below": np.nanmean([u_below, d_below]),
                      "avg_total": np.nanmean([u_avg, d_avg])},
        }

    def _return_vs_benchmark_bins(m_port: pd.Series, m_bench: pd.Series, step=0.01) -> pd.DataFrame:
        if m_port is None or m_bench is None: return pd.DataFrame()
        j = m_port.dropna().index.intersection(m_bench.dropna().index)
        df = pd.DataFrame({"bench": m_bench.reindex(j), "port": m_port.reindex(j)}).dropna()
        bins = np.arange(np.floor(df["bench"].min()/step)*step, np.ceil(df["bench"].max()/step)*step + step, step)
        df["bin"] = pd.cut(df["bench"], bins=bins, include_lowest=True)
        g = df.groupby("bin", observed=True).agg({"bench":"mean", "port":"mean"}).dropna().reset_index()
        g["label"] = g["bin"].astype(str)
        return g

    # ---------- 1. Portfolio Analysis Results ----------
    st.header(f"1. Portfolio Analysis Results ({gs.date()} - {ge.date()})")

    # 보조 함수
    def _kpi_badge(text, val_str, color="#16a34a", bench_str=None):
        bench_html = f'<div style="font-size:12px;color:#6b7280;margin-top:6px;">벤치마크: {bench_str}</div>' if bench_str is not None else ""
        st.markdown(
            f"""
            <div style="display:flex;flex-direction:column;gap:6px;padding:10px 12px;border:1px solid #e5e7eb;border-radius:10px;width:100%;background:#fafafa;">
              <div style="font-size:12px;color:#6b7280">{text}</div>
              <div style="font-weight:700;padding:8px 10px;border-radius:8px;background:{'#fee2e2' if color=='#dc2626' else '#dcfce7'};color:{color};text-align:center;">
                {val_str}
              </div>
              {bench_html}
            </div>
            """, unsafe_allow_html=True
        )
    def _best_worst_year(series: pd.Series):
        y=series.resample("Y").last().pct_change().dropna()
        if y.empty: return None,None
        return (y.idxmax().year, float(y.max())), (y.idxmin().year, float(y.min()))
    def _mdd_details(values: pd.Series):
        v=values.astype(float).dropna()
        if v.size<2: return np.nan,None,None,None,None
        run_max=v.cummax(); dd=v/run_max-1.0
        trough=dd.idxmin(); mdd=float(dd.loc[trough])
        start=run_max.loc[:trough].idxmax()
        rec=v.loc[trough:].loc[lambda s: s>=run_max.loc[start]].first_valid_index()
        down=_month_diff(start,trough); recm=_month_diff(trough,rec) if rec is not None else None
        return mdd,start,trough,rec,(down,recm)

    # 포트폴리오 카드
    for pf_name,pf_struct in st.session_state.portfolios.items():
        s=values_dict.get(pf_name)
        if s is None or s.empty:
            st.info(f"'{pf_name}' 결과가 없습니다."); continue
        m_per_year={"일별":252,"주별":52,"월별":12,"연별":1}[data_freq]
        if st.session_state.rf_mode=="고정값 (%)": rf_input=st.session_state.rf_fixed_value/100.0
        elif st.session_state.rf_mode=="CSV 업로드 (^IRX 등)" and st.session_state.rf_series_data is not None:
            rf_input=st.session_state.rf_series_data
        else: rf_input=0.0
        m_bench=None
        if bench_values is not None and not bench_values.empty:
            b=bench_values.loc[s.index.min():s.index.max()].dropna()
            if not b.empty:
                m_bench=calculate_metrics_v4(b, float(st.session_state.initial_capital), m_per_year, rf_input, None)
        mcalc=calculate_metrics_v4(s, float(st.session_state.initial_capital), m_per_year, rf_input, bench_values)
        st.subheader(pf_name)
        c1,c2=st.columns([2,1], gap="large")
        rows=[]
        for t,meta in pf_struct.items():
            w=float(meta.get("w",0.0))
            if w>0: rows.append({"Ticker":t,"Name":t,"Allocation":f"{w:.2f}%","환노출":bool(meta.get("fx",False))})
        alloc_df=pd.DataFrame(rows)
        with c1:
            if alloc_df.empty: st.info("비중이 0%를 초과하는 자산이 없습니다.")
            else: st.dataframe(alloc_df, hide_index=True, use_container_width=True)
        with c2:
            if not alloc_df.empty:
                vals=[float(str(x).replace('%','')) for x in alloc_df["Allocation"]]
                fig_pie=px.pie(alloc_df, values=vals, names=alloc_df["Name"], hole=0.35, title="자산 비중")
                fig_pie.update_layout(margin=dict(l=10,r=10,t=30,b=70),
                                      legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5))
                st.plotly_chart(fig_pie, use_container_width=True, key=f"alloc_pie_{pf_name}")

        st.markdown("#### Highlights")
        k1,k2,k3=st.columns(3)
        b_cagr=f"{m_bench['CAGR']:.2%}" if m_bench is not None else None
        b_vol=f"{m_bench['Vol']:.2%}" if m_bench is not None else None
        b_mdd=f"{m_bench['MDD']:.2%}" if m_bench is not None else None
        with k1: _kpi_badge("Annualized Return (CAGR)", f"{mcalc['CAGR']:.2%}", "#16a34a", bench_str=b_cagr)
        with k2: _kpi_badge("Standard Deviation", f"{mcalc['Vol']:.2%}", "#dc2626", bench_str=b_vol)
        with k3: _kpi_badge("Maximum Drawdown", f"{mcalc['MDD']:.2%}", "#dc2626", bench_str=b_mdd)

        init_v=float(st.session_state.initial_capital); end_v=float(mcalc["Final"])
        cum_ret=(end_v/init_v-1.0) if init_v!=0 else np.nan
        y_best,y_worst=_best_worst_year(s)
        best_txt=(f"{y_best[0]}년 {y_best[1]:.2%}" if y_best and y_best[0] is not None else "정보 없음")
        worst_txt=(f"{y_worst[0]}년 {y_worst[1]:.2%}" if y_worst and y_worst[0] is not None else "정보 없음")
        bm_growth=""
        if m_bench is not None:
            b=bench_values.loc[s.index.min():s.index.max()].dropna()
            b_init=float(b.iloc[0]); b_end=float(b.iloc[-1]); b_cum=(b_end/b_init-1.0) if b_init!=0 else np.nan
            bm_growth=f" 같은 기간 벤치마크의 최종 가치는 {sym}{b_end:,.0f}이며, 누적수익률은 {b_cum:.2%}입니다."
        st.write(f"{gs.date()}에 {sym}{init_v:,.0f}을 투자했다면 {ge.date()} 기준 가치가 {sym}{end_v:,.0f}로 누적수익률은 {cum_ret:.2%}입니다." + bm_growth)
        st.write(f"최고의 해: {best_txt} · 최악의 해: {worst_txt} · Sharpe: {mcalc['Sharpe']:.2f}" + ("" if m_bench is None else f" · 벤치마크 Sharpe: {m_bench['Sharpe']:.2f}"))
        st.markdown("---")

    # ---------- 2. Performance Summary ----------
    st.header("2. Performance Summary")
    cols=list(values_dict.keys())
    has_any = bool(cols) or (bench_values is not None and not bench_values.empty)
    if has_any:
        data={}
        m_per_year={"일별":252,"주별":52,"월별":12,"연별":1}[data_freq]
        if st.session_state.rf_mode=="고정값 (%)": rf_input=st.session_state.rf_fixed_value/100.0
        elif st.session_state.rf_mode=="CSV 업로드 (^IRX 등)" and st.session_state.rf_series_data is not None: rf_input=st.session_state.rf_series_data
        else: rf_input=0.0

        def _best_worst(series):
            y=series.resample("Y").last().pct_change().dropna()
            return ("N/A","N/A") if y.empty else (f"{y.idxmax().year} ({y.max():.2%})", f"{y.idxmin().year} ({y.min():.2%})")

        # ▶ 벤치마크 행 추가
        if bench_values is not None and not bench_values.empty:
            mb = calculate_metrics_v4(bench_values, float(st.session_state.initial_capital), m_per_year, rf_input, None)
            by, wy = _best_worst(bench_values)
            data[f"벤치마크 ({st.session_state.benchmark_ticker})"] = {
                "Start Balance": f"{sym}{float(st.session_state.initial_capital):,.0f}",
                "End Balance":   f"{sym}{mb['Final']:,.0f}",
                "Annualized Return (CAGR)": f"{mb['CAGR']:.2%}",
                "Standard Deviation": f"{mb['Vol']:.2%}",
                "Best Year": by, "Worst Year": wy,
                "Maximum Drawdown": f"{mb['MDD']:.2%}",
                "Sharpe Ratio": f"{mb['Sharpe']:.2f}" if pd.notna(mb["Sharpe"]) else "N/A",
                "Sortino Ratio": f"{mb['Sortino']:.2f}" if pd.notna(mb["Sortino"]) else "N/A",
            }

        # ▶ 포트폴리오 행들
        for name,s in values_dict.items():
            m=calculate_metrics_v4(s, float(st.session_state.initial_capital), m_per_year, rf_input, None)
            by,wy=_best_worst(s)
            data[name]={"Start Balance":f"{sym}{float(st.session_state.initial_capital):,.0f}",
                        "End Balance":f"{sym}{m['Final']:,.0f}","Annualized Return (CAGR)":f"{m['CAGR']:.2%}",
                        "Standard Deviation":f"{m['Vol']:.2%}","Best Year":by,"Worst Year":wy,
                        "Maximum Drawdown":f"{m['MDD']:.2%}","Sharpe Ratio":f"{m['Sharpe']:.2f}" if pd.notna(m["Sharpe"]) else "N/A",
                        "Sortino Ratio":f"{m['Sortino']:.2f}" if pd.notna(m["Sortino"]) else "N/A"}
        st.dataframe(pd.DataFrame(data), use_container_width=True)
    else:
        st.info("요약을 구성할 포트폴리오가 없습니다.")

    # ---------- 3. Portfolio Growth ----------
    st.header("3. Portfolio Growth")
    log_scale=bool(st.session_state.get("growth_log", False))
    rf_adjust=bool(st.session_state.get("growth_rfadj", False))
    def _rf_growth_series(index: pd.DatetimeIndex, rf_input):
        if isinstance(rf_input,(int,float,np.floating)):
            ann=float(rf_input)
            if ann<=0: return pd.Series(1.0, index=index)
            per={"일별":252,"주별":52,"월별":12,"연별":1}[data_freq]
            r=(1.0+ann)**(1.0/per)-1.0
            return pd.Series((1.0+r), index=index).cumprod()
        elif isinstance(rf_input,pd.Series):
            srf=rf_input.copy(); srf.index=pd.to_datetime(srf.index)
            freq={"일별":"D","주별":"W","월별":"M","연별":"Y"}[data_freq]
            srf=srf.resample(freq).last() if freq!="D" else srf
            per={"일별":252,"주별":52,"월별":12,"연별":1}[data_freq]
            r=((1.0+(srf/100.0))**(1.0/per)-1.0).reindex(index).fillna(0.0)
            return (1.0+r).cumprod()
        else: return pd.Series(1.0, index=index)
    rf_input_for_growth = (st.session_state.rf_fixed_value/100.0 if st.session_state.rf_mode=="고정값 (%)"
                           else (st.session_state.rf_series_data if st.session_state.rf_mode=="CSV 업로드 (^IRX 등)" else 0.0))

    fig_all=go.Figure()
    for name2,s2 in values_dict.items():
        v=s2.loc[gs:ge].dropna()
        if rf_adjust:
            g=_rf_growth_series(v.index, rf_input_for_growth)
            v=(v/v.iloc[0])/(g/float(g.iloc[0]))*float(st.session_state.initial_capital)
        fig_all.add_trace(go.Scatter(x=v.index, y=v.values, mode="lines", name=name2))

    # ▶ 벤치마크 라인 추가
    if bench_values is not None and not bench_values.empty:
        vb = bench_values.loc[gs:ge].dropna()
        if not vb.empty:
            if rf_adjust:
                gb=_rf_growth_series(vb.index, rf_input_for_growth)
                vb=(vb/vb.iloc[0])/(gb/float(gb.iloc[0]))*float(st.session_state.initial_capital)
            fig_all.add_trace(go.Scatter(x=vb.index, y=vb.values, mode="lines", name="Benchmark"))

    fig_all.update_layout(height=440, margin=dict(l=20,r=20,t=30,b=80), hovermode="x unified",
                          yaxis_title=f"가치({sym})", yaxis_tickprefix=sym, yaxis_tickformat=",.0f",
                          yaxis_type=("log" if log_scale else "linear"),
                          legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5))
    st.plotly_chart(fig_all, use_container_width=True, config={"displaylogo": False}, key="growth_all_global")
    copt1,copt2=st.columns([1,1])
    with copt1: st.checkbox("Logarithmic scale", value=log_scale, key="growth_log")
    with copt2: st.checkbox("Risk-free adjusted (excess value)", value=rf_adjust, key="growth_rfadj")

    # ---------- 4. Annual Returns(막대 그래프) ----------
    st.header("4. Annual Returns (막대 그래프)")
    ann_df_list = []
    for name, s in values_dict.items():
        y = s.resample("Y").last().pct_change().dropna()
        if not y.empty:
            ann_df_list.append(pd.DataFrame({"Year": y.index.year, "Return": y.values, "Portfolio": name}))

    # ▶ 벤치마크 열 추가
    if _yret_bench is not None and not _yret_bench.empty:
        ann_df_list.append(pd.DataFrame({"Year": _yret_bench.index.year, "Return": _yret_bench.values, "Portfolio": "Benchmark"}))

    if ann_df_list:
        ann_plot = pd.concat(ann_df_list, axis=0).sort_values("Year")
        fig_ann = px.bar(ann_plot, x="Year", y="Return", color="Portfolio", barmode="group",
                        text=ann_plot["Return"].map(lambda x: f"{x:.1%}"))
        fig_ann.update_traces(textposition="outside", cliponaxis=False)
        fig_ann.update_layout(height=420, margin=dict(l=20,r=20,t=30,b=80),
                              yaxis_tickformat=".0%",
                              legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5))
        st.plotly_chart(fig_ann, use_container_width=True, config={"displaylogo": False}, key="annual_group")
    else:
        st.info("연간 수익률을 계산할 데이터가 없습니다.")

    # ---------- 5. Trailing Returns ----------
    st.header("5. Trailing Returns")
    if _mret_ports or (_mret_bench is not None and not _mret_bench.empty):
        rows = []

        # ▶ 벤치마크 행 추가(있을 경우)
        if _mret_bench is not None and not _mret_bench.empty:
            m = _mret_bench
            row = {"Name": "Benchmark"}
            tr_3m  = _total_return_over_months(m, 3)
            tr_ytd = _ytd_total_return(m)
            tr_1y  = _total_return_over_months(m, 12)
            ar_3y  = _ann_return_from_monthly(m, 3)
            ar_5y  = _ann_return_from_monthly(m, 5)
            ar_full = (1.0 + np.prod(1.0 + m) - 1.0) ** (12.0/len(m)) - 1.0 if len(m)>0 else None
            sd_3y  = _ann_std_from_monthly(m, 3)
            sd_5y  = _ann_std_from_monthly(m, 5)
            row.update({
                ("Total Return","3 Month"): f"{tr_3m:.2%}" if tr_3m is not None else "N/A",
                ("Total Return","Year To Date"): f"{tr_ytd:.2%}" if tr_ytd is not None else "N/A",
                ("Total Return","1 year"): f"{tr_1y:.2%}" if tr_1y is not None else "N/A",
                ("Annualized Return","3 year"): f"{ar_3y:.2%}" if ar_3y is not None else "N/A",
                ("Annualized Return","5 year"): f"{ar_5y:.2%}" if ar_5y is not None else "N/A",
                ("Annualized Return","Full"): f"{ar_full:.2%}" if ar_full is not None else "N/A",
                ("Annualized Standard Deviation","3 year"): f"{sd_3y:.2%}" if sd_3y is not None else "N/A",
                ("Annualized Standard Deviation","5 year"): f"{sd_5y:.2%}" if sd_5y is not None else "N/A",
            })
            rows.append(row)

        # ▶ 포트폴리오 행들
        for pname, m in _mret_ports.items():
            row = {"Name": pname}
            tr_3m  = _total_return_over_months(m, 3)
            tr_ytd = _ytd_total_return(m)
            tr_1y  = _total_return_over_months(m, 12)
            ar_3y  = _ann_return_from_monthly(m, 3)
            ar_5y  = _ann_return_from_monthly(m, 5)
            ar_full = (1.0 + np.prod(1.0 + m) - 1.0) ** (12.0/len(m)) - 1.0 if len(m)>0 else None
            sd_3y  = _ann_std_from_monthly(m, 3)
            sd_5y  = _ann_std_from_monthly(m, 5)
            row.update({
                ("Total Return","3 Month"): f"{tr_3m:.2%}" if tr_3m is not None else "N/A",
                ("Total Return","Year To Date"): f"{tr_ytd:.2%}" if tr_ytd is not None else "N/A",
                ("Total Return","1 year"): f"{tr_1y:.2%}" if tr_1y is not None else "N/A",
                ("Annualized Return","3 year"): f"{ar_3y:.2%}" if ar_3y is not None else "N/A",
                ("Annualized Return","5 year"): f"{ar_5y:.2%}" if ar_5y is not None else "N/A",
                ("Annualized Return","Full"): f"{ar_full:.2%}" if ar_full is not None else "N/A",
                ("Annualized Standard Deviation","3 year"): f"{sd_3y:.2%}" if sd_3y is not None else "N/A",
                ("Annualized Standard Deviation","5 year"): f"{sd_5y:.2%}" if sd_5y is not None else "N/A",
            })
            rows.append(row)

        st.dataframe(pd.DataFrame(rows).set_index("Name"), use_container_width=True)
    else:
        st.info("트레일링 수익률을 계산할 월간 수익률이 없습니다.")

    # ---------- 6. Annualized Active Return ----------
    st.header("6. Annualized Active Return")
    if _yret_bench is None or _yret_bench.empty:
        st.info("벤치마크 연간 수익률이 없어 액티브 리턴을 계산할 수 없습니다.")
    else:
        frames=[]
        for pname, y in _yret_ports.items():
            j = y.index.intersection(_yret_bench.index)
            if len(j)==0:
                continue
            diff = (y.reindex(j) - _yret_bench.reindex(j)).dropna()
            frames.append(pd.DataFrame({"Year": j.year, "Active": diff.values, "Portfolio": pname}))
        if frames:
            df_act = pd.concat(frames, axis=0)
            fig_ar = px.bar(df_act, x="Year", y="Active", color="Portfolio",
                            barmode="group", text=df_act["Active"].map(lambda x: f"{x:.1%}"))
            fig_ar.update_traces(textposition="outside", cliponaxis=False)
            # ▼ v5.4.1 변경: 범례를 아래로 이동
            fig_ar.update_layout(height=420, margin=dict(l=20,r=20,t=30,b=80),
                                 yaxis_tickformat=".0%",
                                 legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5))
            st.plotly_chart(fig_ar, use_container_width=True, key="annual_active_ret")
        else:
            st.info("액티브 리턴을 표시할 연간 교집합 데이터가 없습니다.")

    # ---------- 7. Active Return Contribution ----------
    st.header("7. Active Return Contribution")

    if _mret_bench is None or _mret_bench.empty:
        st.info("벤치마크 월간 수익률이 없어 액티브 기여도를 계산할 수 없습니다.")
    else:
        # st.session_state.portfolios: {포트명: {티커: {w:비중, name:자산명, fx:bool?}, ...}}
        for pname, pstruct in st.session_state.portfolios.items():
            m_port = _mret_ports.get(pname, pd.Series(dtype=float))
            if m_port is None or m_port.empty:
                continue

            # 공통 월말 인덱스
            j = m_port.index.intersection(_mret_bench.index)
            if len(j) == 0:
                continue
            bench = _mret_bench.reindex(j)

            # 1) 포트폴리오 목표 비중(자산 라벨) 벡터 구성
            #    - v5.4.2에서 사용한 라벨 규칙 그대로: USD가 기본이면서 fx=False면 '(H)' 미부착,
            #      base_currency != 'USD' 이고 fx=False면 '(H)' 접미사 부착
            all_assets = _mret_assets.columns
            W = pd.Series(0.0, index=all_assets, dtype=float)
            tot = 0.0
            for t, meta in pstruct.items():
                label = f"{t}{'(H)' if (st.session_state.base_currency!='USD' and not bool(meta.get('fx', False))) else ''}"
                if label in W.index:
                    w = float(meta.get("w", 0.0))
                    if w > 0:
                        W[label] += w
                        tot += w
            if tot <= 0:
                continue
            W = (W / tot)
            W_nz = W[W > 0].copy()  # 사용하지 않은 자산 제외

            # 2) 자산별 월간 액티브 기여도: w_i * (r_i - r_bench)
            A = {}
            for a in W_nz.index:
                r_i = _mret_assets[a].reindex(j)
                A[a] = (W_nz[a] * (r_i - bench)).rename(a)
            dfA = pd.concat(A.values(), axis=1).fillna(0.0)

            # 누적(시점까지 합계) → 누적 스택 막대
            dfA_cum = dfA.cumsum()

            st.subheader(pname)
            df_plot = (
                dfA_cum.reset_index()
                      .melt(id_vars=["index"], var_name="Asset", value_name="ActiveContrib")
                      .rename(columns={"index": "Date"})
            )
            fig_ac = px.bar(
                df_plot, x="Date", y="ActiveContrib", color="Asset", barmode="relative"
            )
            fig_ac.update_traces(hovertemplate="%{y:.2%}<extra></extra>")
            fig_ac.add_trace(
                go.Scatter(
                    x=(m_port.reindex(j) - bench).cumsum().index,
                    y=(m_port.reindex(j) - bench).cumsum().values,
                    mode="lines",
                    name="Total (Portfolio − Benchmark)",
                    line=dict(width=2),
                )
            )
            fig_ac.update_layout(
                height=420,
                margin=dict(l=20, r=20, t=30, b=80),
                yaxis_tickformat=".1%",
                hovermode="x unified",           # ← 마우스 올리면 자산별 결과 한 번에 표시
                legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5),
                title="Cumulative Active Return Contribution (stacked)",
            )
            st.plotly_chart(fig_ac, use_container_width=True, key=f"active_contrib_{pname}")

            # 3) 1·3·5년 합산 기여도 표(전치: 행=자산, 열=기간)
            periods = [12, 36, 60]
            labels = ["1Y", "3Y", "5Y"]
            port_active = (m_port.reindex(j) - bench)
            agg = {}     # {asset: {"1Y":val, "3Y":val, "5Y":val}}
            totals = {}  # {"1Y": 총합, ...}

            for mths, lab in zip(periods, labels):
                if len(dfA) >= mths:
                    seg = dfA.tail(mths)
                    seg_sum = seg.sum(axis=0)      # 자산별 합(퍼센트포인트)
                    totals[lab] = port_active.tail(mths).sum()
                    for a in W_nz.index:
                        agg.setdefault(a, {})[lab] = float(seg_sum.get(a, 0.0))

            if agg:
                disp = pd.DataFrame(agg).T  # 자산 행, 기간 열
                for c in disp.columns:
                    disp[c] = disp[c].map(lambda x: f"{x:.2%}")
                if totals:
                    disp.loc["Total (Portfolio − Benchmark)"] = {k: f"{v:.2%}" for k, v in totals.items()}
                st.dataframe(disp, use_container_width=True)

    # ---------- 8. Rolling Active Return ----------
    st.header("8. Rolling Active Return")
    if _mret_bench is None or _mret_bench.empty:
        st.info("벤치마크 월간 수익률이 없어 롤링 액티브 리턴을 계산할 수 없습니다.")
    else:
        win = st.selectbox("Window (months)", options=[12, 24, 36, 60], index=2, key="rar_window")
        for pname, m in _mret_ports.items():
            m_act = _active_monthly(m, _mret_bench)
            if m_act.empty:
                continue
            bar = _rolling_active_ann(m_act, int(win))       # 좌축(막대)
            te  = _tracking_error(m_act, int(win))           # 우측(선)

            fig = go.Figure()
            fig.add_trace(go.Bar(x=bar.index, y=bar.values, name="Active Return"))
            fig.add_trace(go.Scatter(x=te.index,  y=te.values, name="Tracking Error", mode="lines", yaxis="y2"))
            fig.update_layout(
                height=420, margin=dict(l=20,r=20,t=30,b=80),
                yaxis=dict(title="Active Return", tickformat=".1%"),
                yaxis2=dict(title="Tracking Error", overlaying="y", side="right", tickformat=".1%"),
                title=f"{pname} — Rolling Active Return and Risk ({int(win)} months)",
                legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5)
            )
            st.plotly_chart(fig, use_container_width=True, key=f"rolling_active_{pname}")

    # ---------- 9. Up vs. Down Market Performance ----------
    st.header("9. Up vs. Down Market Performance")
    if _mret_bench is None or _mret_bench.empty:
        st.info("벤치마크 월간 수익률이 없어 업/다운 성과를 계산할 수 없습니다.")
    else:
        for pname, m in _mret_ports.items():
            stat = _up_down_table(m, _mret_bench)
            if not stat:
                continue
            df = pd.DataFrame({
                "Market Type": ["Up Market", "Down Market", "Total"],
                "Above Benchmark": [stat["up"]["above"], stat["down"]["above"], stat["total"]["above"]],
                "Below Benchmark": [stat["up"]["below"], stat["down"]["below"], stat["total"]["below"]],
                "Total": [stat["up"]["total"], stat["down"]["total"], stat["total"]["total"]],
                "% Above Benchmark": [
                    f"{(stat['up']['above']/stat['up']['total']*100):.0f}%" if stat['up']['total']>0 else 'N/A',
                    f"{(stat['down']['above']/stat['down']['total']*100):.0f}%" if stat['down']['total']>0 else 'N/A',
                    f"{(stat['total']['above']/stat['total']['total']*100):.0f}%" if stat['total']['total']>0 else 'N/A',
                ],
                "Average Active Return Above Benchmark": [
                    f"{stat['up']['avg_above']:.2%}" if pd.notna(stat['up']['avg_above']) else "N/A",
                    f"{stat['down']['avg_above']:.2%}" if pd.notna(stat['down']['avg_above']) else "N/A",
                    f"{stat['total']['avg_above']:.2%}" if pd.notna(stat['total']['avg_above']) else "N/A",
                ],
                "Average Active Return Below Benchmark": [
                    f"{stat['up']['avg_below']:.2%}" if pd.notna(stat['up']['avg_below']) else "N/A",
                    f"{stat['down']['avg_below']:.2%}" if pd.notna(stat['down']['avg_below']) else "N/A",
                    f"{stat['total']['avg_below']:.2%}" if pd.notna(stat['total']['avg_below']) else "N/A",
                ],
                "Average Active Return Total": [
                    f"{stat['up']['avg_total']:.2%}" if pd.notna(stat['up']['avg_total']) else "N/A",
                    f"{stat['down']['avg_total']:.2%}" if pd.notna(stat['down']['avg_total']) else "N/A",
                    f"{stat['total']['avg_total']:.2%}" if pd.notna(stat['total']['avg_total']) else "N/A",
                ],
            })
            st.subheader(pname)
            st.dataframe(df, use_container_width=True)

            bins_df = _return_vs_benchmark_bins(m, _mret_bench, step=0.01)
            if not bins_df.empty:
              # ===== Return vs. Benchmark (정렬 순번 X축, 눈금은 벤치마크 % 라벨) =====
              df = pd.DataFrame({"Benchmark": _mret_bench, pname: m_port}).dropna()
              if not df.empty:
                  # 1) 벤치마크 수익률 기준 오름차순 정렬
                  df = df.sort_values("Benchmark").reset_index(drop=True)

                  # 2) X축: 고유 정수 인덱스(중복 없음), ticktext는 벤치마크 퍼센트 라벨
                  xs = np.arange(len(df))
                  ticktext = df["Benchmark"].map(lambda x: f"{x:+.1%}").tolist()

                  # 3) 그룹 막대(포트폴리오 vs 벤치마크)
                  fig_rvb = go.Figure()
                  fig_rvb.add_bar(x=xs, y=df[pname].values, name=pname)
                  fig_rvb.add_bar(x=xs, y=df["Benchmark"].values, name="Benchmark")

                  # 4) 축/레이아웃
                  #    - tick 간격이 많을 때 라벨 과밀을 줄이기 위해 표시 간격(step)을 조절
                  step = max(1, len(xs) // 50)   # 한 화면에 ~50개 라벨만 보이게
                  show_idx = xs[::step]
                  show_lbl = [ticktext[i] for i in show_idx]

                  fig_rvb.update_xaxes(
                      tickmode="array",
                      tickvals=show_idx,
                      ticktext=show_lbl,
                      title_text="Benchmark Return"
                  )
                  fig_rvb.update_yaxes(tickformat=".1%", title_text="Return")

                  fig_rvb.update_traces(hovertemplate="%{y:.2%}<extra></extra>")
                  fig_rvb.update_layout(
                      barmode="group",
                      hovermode="x unified",
                      height=520,
                      margin=dict(l=20, r=20, t=40, b=90),
                      legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5),
                  )

                  st.plotly_chart(fig_rvb, use_container_width=True, key=f"updown_rvb_{pname}")

    # ---------- 10. Risk and Return Metrics ----------
    st.header("10. Risk and Return Metrics")

    # 보조: 월간 Rf 시리즈/수치로 변환
    def _rf_monthly_series(index: pd.DatetimeIndex, rf_input):
        if isinstance(rf_input,(int,float,np.floating)):
            ann=float(rf_input)
            r_m=(1.0+ann)**(1.0/12)-1.0
            return pd.Series(r_m, index=index)
        elif isinstance(rf_input,pd.Series):
            srf=rf_input.copy(); srf.index=pd.to_datetime(srf.index)
            srf=srf.resample("M").last()/100.0
            r_m=((1.0+srf)**(1.0/12)-1.0)
            return r_m.reindex(index).ffill().bfill().fillna(0.0)
        else:
            return pd.Series(0.0, index=index)

    def _cagr_from_values(v: pd.Series):
        v=v.dropna()
        if v.size<2: return np.nan
        n_years=(v.index[-1].to_period('M') - v.index[0].to_period('M'))/12
        n_years=float(n_years) if n_years!=0 else (len(v)/252.0)
        return (float(v.iloc[-1]/v.iloc[0])**(1.0/max(n_years,1e-9)))-1.0

    def _risk_return_table(values_dict, bench_values, rf_input):
        rows=[]
        bm=None
        if bench_values is not None:
            bm = bench_values.resample("M").last().pct_change().dropna()
        for name, s in values_dict.items():
            pm = s.resample("M").last().pct_change().dropna()
            if pm.empty: continue
            idx = pm.index
            rf_m = _rf_monthly_series(idx, rf_input)

            mu_m = float(pm.mean())
            mu_a_arith = mu_m * 12.0
            gm_m = (np.prod(1.0+pm)**(1.0/len(pm)) - 1.0) if len(pm)>0 else np.nan
            gm_a = (1.0+gm_m)**12 - 1.0 if pd.notna(gm_m) else np.nan

            sd_m = float(pm.std(ddof=1))
            sd_a = sd_m * sqrt(12.0)
            downside = np.minimum(0.0, pm)
            dd_m = float(downside.std(ddof=1))

            ex_m = pm - rf_m
            sharpe = (float(ex_m.mean())/float(sd_m))*sqrt(12.0) if sd_m>0 else np.nan
            sortino = (float(ex_m.mean())/float(dd_m))*sqrt(12.0) if dd_m>0 else np.nan

            mdd, calmar = np.nan, np.nan
            try:
                v=s.loc[idx.min():idx.max()].dropna()
                runmax=v.cummax()
                dd=v/runmax-1.0
                mdd=float(dd.min())
                cagr=_cagr_from_values(v)
                calmar = (cagr/abs(mdd)) if (pd.notna(cagr) and mdd<0) else np.nan
            except Exception:
                pass

            corr=beta=alpha_a=r2=treynor=m2=active=te=ir=upcap=downcap=np.nan
            rf_a = (1.0+rf_m.mean())**12 - 1.0 if len(rf_m)>0 else 0.0
            if bm is not None and not bm.empty:
                df = pd.concat([pm.rename("p"), bm.rename("b")], axis=1).dropna()
                if not df.empty and df["b"].std(ddof=1)>0:
                    corr = float(df["p"].corr(df["b"]))
                    var_b = float(df["b"].var(ddof=1))
                    beta = float(df["p"].cov(df["b"])/var_b) if var_b>0 else np.nan
                    alpha_m = float((df["p"]-rf_m.reindex(df.index)).mean() - beta*(df["b"]-rf_m.reindex(df.index)).mean()) if pd.notna(beta) else np.nan
                    alpha_a = alpha_m*12.0 if pd.notna(alpha_m) else np.nan
                    r2 = corr**2 if pd.notna(corr) else np.nan
                    treynor = ((gm_a - rf_a)/beta*100.0) if (pd.notna(beta) and beta!=0 and pd.notna(gm_a)) else np.nan
                    sd_b_a = float(df["b"].std(ddof=1))*sqrt(12.0)
                    m2 = rf_a + sharpe*sd_b_a if (pd.notna(sharpe) and sd_b_a>0) else np.nan
                    active = gm_a - ((1.0+float(df["b"].mean()))**12 - 1.0)
                    diff = df["p"] - df["b"]
                    te = float(diff.std(ddof=1))*sqrt(12.0)
                    ir = (active/te) if (pd.notna(active) and te and te>0) else np.nan
                    up = df[df["b"]>0.0]; down=df[df["b"]<0.0]
                    def _gm(s): return (np.prod(1.0+s)**(1.0/len(s)) - 1.0) if len(s)>0 else np.nan
                    upcap = (100.0*(_gm(up["p"])/max(_gm(up["b"]),1e-12))) if len(up)>0 and pd.notna(_gm(up["p"])) and pd.notna(_gm(up["b"])) else np.nan
                    downcap = (100.0*(_gm(down["p"])/max(_gm(down["b"]),1e-12))) if len(down)>0 and pd.notna(_gm(down["p"])) and pd.notna(_gm(down["b"])) else np.nan

            skew = float(pm.skew())
            ex_kurt = float(pm.kurt())
            hist_var5 = -float(np.nanpercentile(pm, 5))
            sigma = float(pm.std(ddof=1)); mu=float(pm.mean()); z = norm.ppf(0.05)
            anal_var5 = -(mu + z*sigma) if np.isfinite(sigma) else np.nan
            cvar5 = -float(pm[pm <= np.nanpercentile(pm,5)].mean()) if (pm<=np.nanpercentile(pm,5)).any() else np.nan

            pos_periods = int((pm>0).sum())
            gl_ratio = (float(pm[pm>0].mean())/abs(float(pm[pm<0].mean()))) if (pm<0).any() and (pm>0).any() else np.nan

            rows.append({
                "Portfolio": name,
                "Arithmetic Mean (monthly)": f"{mu_m:.3%}",
                "Arithmetic Mean (annualized)": f"{mu_a_arith:.3%}",
                "Geometric Mean (monthly)": (f"{gm_m:.3%}" if pd.notna(gm_m) else "N/A"),
                "Geometric Mean (annualized)": (f"{gm_a:.3%}" if pd.notna(gm_a) else "N/A"),
                "Standard Deviation (monthly)": f"{sd_m:.3%}",
                "Standard Deviation (annualized)": f"{sd_a:.3%}",
                "Downside Deviation (monthly)": (f"{dd_m:.3%}" if pd.notna(dd_m) else "N/A"),
                "Maximum Drawdown": f"{mdd:.2%}" if pd.notna(mdd) else "N/A",
                "Benchmark Correlation": (f"{corr:.2f}" if pd.notna(corr) else "N/A"),
                "Beta(*)": (f"{beta:.2f}" if pd.notna(beta) else "N/A"),
                "Alpha (annualized)": (f"{alpha_a:.2%}" if pd.notna(alpha_a) else "N/A"),
                "R2": (f"{r2:.2f}" if pd.notna(r2) else "N/A"),
                "Sharpe Ratio": (f"{sharpe:.2f}" if pd.notna(sharpe) else "N/A"),
                "Sortino Ratio": (f"{sortino:.2f}" if pd.notna(sortino) else "N/A"),
                "Treynor Ratio (%)": (f"{treynor:.2f}" if pd.notna(treynor) else "N/A"),
                "Calmar Ratio": (f"{calmar:.2f}" if pd.notna(calmar) else "N/A"),
                "Modigliani–Modigliani Measure": (f"{m2:.2%}" if pd.notna(m2) else "N/A"),
                "Active Return": (f"{active:.2%}" if pd.notna(active) else "N/A"),
                "Tracking Error": (f"{te:.2%}" if pd.notna(te) else "N/A"),
                "Information Ratio": (f"{ir:.2f}" if pd.notna(ir) else "N/A"),
                "Skewness": f"{skew:.2f}",
                "Excess Kurtosis": f"{ex_kurt:.2f}",
                "Historical Value-at-Risk (5%)": f"{hist_var5:.2%}",
                "Analytical Value-at-Risk (5%)": (f"{anal_var5:.2%}" if pd.notna(anal_var5) else "N/A"),
                "Conditional Value-at-Risk (5%)": (f"{cvar5:.2%}" if pd.notna(cvar5) else "N/A"),
                "Upside Capture Ratio (%)": (f"{upcap:.2f}" if pd.notna(upcap) else "N/A"),
                "Downside Capture Ratio (%)": (f"{downcap:.2f}" if pd.notna(downcap) else "N/A"),
                "Positive Periods": pos_periods,
                "Gain/Loss Ratio": (f"{gl_ratio:.2f}" if pd.notna(gl_ratio) else "N/A"),
            })
        return pd.DataFrame(rows)

    if st.session_state.rf_mode=="고정값 (%)": rf_input_for_metrics=st.session_state.rf_fixed_value/100.0
    elif st.session_state.rf_mode=="CSV 업로드 (^IRX 등)" and st.session_state.rf_series_data is not None:
        rf_input_for_metrics=st.session_state.rf_series_data
    else: rf_input_for_metrics=0.0

    rr_df = _risk_return_table(values_dict, bench_values, rf_input_for_metrics)
    if not rr_df.empty:
        st.dataframe(rr_df.set_index("Portfolio").T, use_container_width=True)
    else:
        st.info("계산할 포트폴리오 수익률이 없습니다.")

    # ---------- 11. Annual Returns(표) ----------
    st.header("11. Annual Returns (표)")
    def _force_year_index(df: pd.DataFrame) -> pd.DataFrame:
        out = df.copy()
        idx = out.index
        if isinstance(idx, pd.MultiIndex):
            new_year = None
            for i in range(idx.nlevels):
                try:
                    dt = pd.to_datetime(idx.get_level_values(i), errors="coerce")
                    if dt.notna().any():
                        new_year = dt.year
                        break
                except Exception:
                    pass
            if new_year is None:
                try:
                    new_year = pd.Index(idx.get_level_values(-1)).astype(int)
                except Exception:
                    new_year = pd.to_datetime(idx.get_level_values(-1), errors="coerce").year
            out.index = pd.Index(new_year, name="Year")
        elif isinstance(idx, pd.DatetimeIndex):
            out.index = pd.Index(idx.year, name="Year")
        else:
            try:
                out.index = pd.Index(out.index.astype(int), name="Year")
            except Exception:
                out.index = pd.to_datetime(out.index, errors="coerce").year
                out.index = pd.Index(out.index, name="Year")
        return out

    frames=[]
    for name,s in values_dict.items():
        yend=s.resample("Y").last()
        yret=yend.pct_change().dropna()
        ybal=yend.dropna().iloc[1:]
        tmp=pd.DataFrame({"Return": yret, "Balance": ybal})
        tmp.columns=pd.MultiIndex.from_product([[name], tmp.columns])
        tmp=_force_year_index(tmp)
        frames.append(tmp)
    ann_tbl=pd.concat(frames, axis=1) if frames else pd.DataFrame()
    if isinstance(df_proc_used, pd.DataFrame) and not df_proc_used.empty:
        asset_y=df_proc_used.resample("Y").last().pct_change().dropna()
        asset_y=_force_year_index(asset_y)
        new_cols=[]
        for col in asset_y.columns:
            if isinstance(col, tuple) and len(col)==2:
                t,fx=col
                label=f"{t}{'(H)' if not fx and st.session_state.base_currency!='USD' else ''}"
                new_cols.append(label)
            else:
                new_cols.append(str(col))
        asset_y.columns=new_cols
        ann_tbl=pd.concat([ann_tbl, asset_y], axis=1)
    st.dataframe(ann_tbl.sort_index(), use_container_width=True)

    # ---------- 12. Monthly Returns(표) ----------
    st.header("12. Monthly Returns (표)")
    frames=[]
    for name,s in values_dict.items():
        mend=s.resample("M").last()
        mret=mend.pct_change()
        mbal=mend
        dfp=pd.DataFrame({
            (name,"Return"): mret.values,
            (name,"Balance"): mbal.values
        }, index=mend.index)
        frames.append(dfp)

    m_tbl=pd.concat(frames, axis=1) if frames else pd.DataFrame()

    if isinstance(df_proc_used, pd.DataFrame) and not df_proc_used.empty:
        asset_m=df_proc_used.resample("M").last().pct_change()
        new_cols=[]
        for (t,fx) in asset_m.columns:
            label=f"{t}{'(H)' if not fx and st.session_state.base_currency!='USD' else ''}"
            new_cols.append(label)
        asset_m.columns=new_cols
        m_tbl=pd.concat([m_tbl, asset_m], axis=1)

    if not m_tbl.empty:
        m_tbl["Year"] = m_tbl.index.year
        m_tbl["Month"] = m_tbl.index.month
        first_cols = ["Year","Month"]
        other_cols = [c for c in m_tbl.columns if c not in first_cols]
        m_tbl = m_tbl[first_cols + other_cols]

    disp=m_tbl.copy()
    for c in disp.columns:
        if isinstance(c, tuple) and len(c)==2 and c[1]=="Return":
            disp[c]=disp[c].map(lambda x: f"{x:.2%}" if pd.notna(x) else "—")
        elif isinstance(c, tuple) and len(c)==2 and c[1]=="Balance":
            disp[c]=disp[c].map(lambda x: f"{sym}{x:,.0f}" if pd.notna(x) else "—")
        elif isinstance(c,str) and c not in ("Year","Month"):
            disp[c]=disp[c].map(lambda x: f"{x:.2%}" if pd.notna(x) else "—")

    sizes=[12,24,36,60,120]
    st.selectbox("Show entries", options=sizes,
                 index=sizes.index(st.session_state.mr_page_size) if st.session_state.mr_page_size in sizes else 0,
                 key="mr_page_size")
    total_rows=len(disp); per_page=int(st.session_state.mr_page_size)
    total_pages=max(1, math.ceil(total_rows/per_page))
    start=(st.session_state.mr_page-1)*per_page; end=start+per_page
    st.caption(f"총 {total_rows}행 · 페이지 {st.session_state.mr_page}/{total_pages}")
    st.dataframe(disp.iloc[start:end], use_container_width=True)
    cnav1,cnav2,_=st.columns([1,1,6])
    with cnav1:
        if st.button("Previous", disabled=(st.session_state.mr_page<=1), key="mr_prev"):
            st.session_state.mr_page=max(1, st.session_state.mr_page-1); st.rerun()
    with cnav2:
        if st.button("Next", disabled=(st.session_state.mr_page>=total_pages), key="mr_next"):
            st.session_state.mr_page=min(total_pages, st.session_state.mr_page+1); st.rerun()

    # ---------- 13. Monthly Return Heatmaps ----------
    st.header("13. Monthly Return Heatmaps")
    with st.expander("색 범위 설정(연·월 공통)", expanded=False):
        c1, c2, c3 = st.columns([1,1,1])
        with c1:
            st.radio("모드", options=("자동","수동"), key="ret_cmap_mode", horizontal=True)
        with c2:
            if st.session_state.ret_cmap_mode == "수동":
                st.number_input("최소(%)", value=(st.session_state.ret_cmap_min if st.session_state.ret_cmap_min is not None else -30.0),
                                step=1.0, key="ret_cmap_min", format="%.2f")
        with c3:
            if st.session_state.ret_cmap_mode == "수동":
                st.number_input("최대(%)", value=(st.session_state.ret_cmap_max if st.session_state.ret_cmap_max is not None else 30.0),
                                step=1.0, key="ret_cmap_max", format="%.2f")
        cc1, cc2 = st.columns([1,1])
        with cc1:
            if st.button("적용", use_container_width=True, key="ret_range_apply_v519"):
                _toast_and_rerun("수익률 색 범위를 적용했습니다.")
        with cc2:
            if st.button("리셋(자동)", type="secondary", use_container_width=True, key="ret_range_reset_v519"):
                st.session_state.ret_cmap_mode = "자동"
                st.session_state.ret_cmap_min = None
                st.session_state.ret_cmap_max = None
                _toast_and_rerun("수익률 색 범위를 자동으로 복귀했습니다.")

    for name, s in values_dict.items():
        mret = s.resample("M").last().pct_change().dropna()
        if mret.empty:
            continue
        rng_m = _get_return_color_range_manual_or_auto(mret)
        dfm = mret.to_frame("ret"); dfm["Year"] = dfm.index.year; dfm["Month"] = dfm.index.month
        pivot = dfm.pivot(index="Year", columns="Month", values="ret").sort_index().reindex(columns=range(1,13))
        st.markdown(f"**{name}**")
        fig_m = px.imshow(
            pivot.values,
            x=[f"{m}월" for m in pivot.columns],
            y=pivot.index.astype(str),
            aspect="auto", origin="upper",
            color_continuous_scale=RET_CMAP,
            range_color=rng_m,
            title=None
        )
        text_vals = np.where(np.isnan(pivot.values), "", np.vectorize(lambda x: f"{x:.1%}")(pivot.values))
        fig_m.update_traces(text=text_vals, texttemplate="%{text}",
                            hovertemplate="연도:%{y}  월:%{x}<br>수익률:%{text}<extra></extra>",
                            selector=dict(type="heatmap"))
        fig_m.update_traces(textfont={"size": 11}, selector=dict(type="heatmap"))
        fig_m.update_layout(margin=dict(l=20,r=20,t=10,b=20), coloraxis_colorbar=dict(title="수익률"))
        st.plotly_chart(fig_m, use_container_width=True, key=f"mheat_{name}")

    # ---------- 14. Drawdowns(차트) ----------
    st.header("14. Drawdowns (차트)")
    fig_dd=go.Figure()
    for name,s in values_dict.items():
        v=s.loc[gs:ge].dropna(); dd=v/v.cummax()-1.0
        fig_dd.add_trace(go.Scatter(x=dd.index, y=dd.values, mode="lines", name=name))
    fig_dd.update_layout(height=420, margin=dict(l=20,r=20,t=30,b=80), yaxis_tickformat=".0%", hovermode="x unified",
                         legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5))
    st.plotly_chart(fig_dd, use_container_width=True, key="dd_chart_all")

    # ---------- 15. Historical Market Stress Periods ----------
    st.header("15. Historical Market Stress Periods")
    st.caption("사용자 정의 스트레스 구간을 입력하면 구간 수익률을 계산합니다. (YYYY-MM 권장)")

    with st.expander("스트레스 구간 입력", expanded=False):
        if "stress_rows" not in st.session_state:
            st.session_state.stress_rows = pd.DataFrame([
                {"Stress Period":"COVID-19 Start","Start":"2020-01","End":"2020-03"}
            ])
        stress_df = st.data_editor(st.session_state.stress_rows, num_rows="dynamic", use_container_width=True)
        st.session_state.stress_rows = stress_df

    def _period_total_return(series: pd.Series, start: str, end: str) -> float | None:
        try:
            sdt = pd.to_datetime(start+"-01") if len(start)==7 else pd.to_datetime(start)
            edt = pd.to_datetime(end+"-01") if len(end)==7 else pd.to_datetime(end)
            x = series.loc[(series.index >= sdt) & (series.index <= edt)]
            if len(x) < 2: return None
            return float(x.iloc[-1]/x.iloc[0] - 1.0)
        except Exception:
            return None

    if not st.session_state.stress_rows.empty:
        out_rows=[]
        for _, r in st.session_state.stress_rows.iterrows():
            name = str(r.get("Stress Period","")); stt=str(r.get("Start","")); end=str(r.get("End",""))
            row = {"Stress Period": name, "Start": stt, "End": end}
            for pname, s in values_dict.items():
                val = _period_total_return(s, stt, end)
                row[pname] = f"{val:.2%}" if val is not None else "N/A"
            if bench_values is not None and not bench_values.empty:
                br = _period_total_return(bench_values, stt, end)
                row["Benchmark"] = f"{br:.2%}" if br is not None else "N/A"
            out_rows.append(row)
        st.dataframe(pd.DataFrame(out_rows), use_container_width=True)

    # ---------- 16. Drawdowns for Portfolio ----------
    st.header("16. Drawdowns for Portfolio")
    def _episodes(values: pd.Series) -> List[dict]:
        v=values.astype(float).dropna()
        if v.size<2: return []
        run_max=v.expanding().max(); dd=v/run_max-1.0
        episodes=[]; peak=v.index[0]; trough=v.index[0]; min_dd=0.0; in_dd=False
        for t in v.index:
            if v.loc[t] >= run_max.loc[t]-1e-12:
                if in_dd: episodes.append({"Start":peak,"Trough":trough,"End":t,"DD":min_dd})
                peak=t; trough=t; min_dd=0.0; in_dd=False
            else:
                in_dd=True
                if dd.loc[t] < min_dd: min_dd=float(dd.loc[t]); trough=t
        if in_dd: episodes.append({"Start":peak,"Trough":trough,"End":None,"DD":min_dd})
        episodes.sort(key=lambda x:x["DD"])
        return episodes
    def _months(a,b):
        if a is None or b is None: return None
        return (b.year-a.year)*12 + (b.month-a.month)
    for name,s in values_dict.items():
        eps=_episodes(s); rows=[]
        for i,e in enumerate(eps[:10],1):
            start, trough, end, dd=e["Start"], e["Trough"], e["End"], e["DD"]
            down_len=_months(start,trough); rec_len=_months(trough,end); total_len=_months(start,end)
            rows.append({"Rank":i,"Start":start.strftime("%b %Y"),"End":trough.strftime("%b %Y"),
                         "Length":f"{down_len} months" if down_len is not None else "—",
                         "Recovery By":(end.strftime("%b %Y") if end is not None else "—"),
                         "Recovery Time":(f"{rec_len} months" if rec_len is not None else "—"),
                         "Underwater Period":(f"{total_len} months" if total_len is not None else "—"),
                         "Drawdown":f"{dd:.2%}"})
        st.subheader(f"Drawdowns for {name}")
        st.dataframe(pd.DataFrame(rows), use_container_width=True)

    # ---------- 17. Portfolio Assets ----------
    st.header("17. Portfolio Assets")
    if _mret_assets is not None and not _mret_assets.empty:
        if st.session_state.rf_mode=="고정값 (%)":
            rf_m = (_rf_monthly_series(_mret_assets.index, st.session_state.rf_fixed_value/100.0)
                    .reindex(_mret_assets.index).fillna(0.0))
        elif st.session_state.rf_mode=="CSV 업로드 (^IRX 등)" and st.session_state.rf_series_data is not None:
            rf_m = (_rf_monthly_series(_mret_assets.index, st.session_state.rf_series_data)
                    .reindex(_mret_assets.index).fillna(0.0))
        else:
            rf_m = pd.Series(0.0, index=_mret_assets.index)

        rows=[]
        for a in _mret_assets.columns:
            m = _mret_assets[a].dropna()
            if m.empty:
                continue
            mu_m = float(m.mean()); sd_m = float(m.std(ddof=1))
            mu_a = mu_m*12.0; sd_a = sd_m*sqrt(12.0)
            downside = np.minimum(0.0, m); dd_m = float(downside.std(ddof=1))
            sharpe = ((m - rf_m).mean()/sd_m)*sqrt(12.0) if sd_m>0 else np.nan
            sortino = ((m - rf_m).mean()/dd_m)*sqrt(12.0) if dd_m>0 else np.nan
            s = _assets[a]
            y = s.resample("Y").last().pct_change().dropna()
            best = f"{y.idxmax().year} ({float(y.max()):.2%})" if not y.empty else "N/A"
            worst = f"{y.idxmin().year} ({float(y.min()):.2%})" if not y.empty else "N/A"
            runmax = s.cummax(); mdd = float((s/runmax-1.0).min()) if len(s)>1 else np.nan
            rows.append({
                "Ticker": a, "Name": a,
                "CAGR": f"{mu_a:.2%}", "Stdev": f"{sd_a:.2%}",
                "Best Year": best, "Worst Year": worst,
                "Max Drawdown": f"{mdd:.2%}" if pd.notna(mdd) else "N/A",
                "Sharpe Ratio": f"{sharpe:.2f}" if pd.notna(sharpe) else "N/A",
                "Sortino Ratio": f"{sortino:.2f}" if pd.notna(sortino) else "N/A",
            })
        st.dataframe(pd.DataFrame(rows), use_container_width=True)
    else:
        st.info("자산 성과를 계산할 월간 데이터가 없습니다.")

    # ---------- 18. Monthly Correlations ----------
    st.header("18. Monthly Correlations")
    mats = []
    if _mret_assets is not None and not _mret_assets.empty:
        mats.append(_mret_assets)
    for pname, m in _mret_ports.items():
        if not m.empty:
            mats.append(m.rename(pname).to_frame())
    # ▶ 벤치마크 열 추가
    if _mret_bench is not None and not _mret_bench.empty:
        mats.append(_mret_bench.rename("Benchmark").to_frame())

    if mats:
        all_m = pd.concat(mats, axis=1).dropna(how="any")
        corr = all_m.corr()
        rng = _get_corr_color_range()
        fig_corr = px.imshow(
            corr.values,
            x=corr.columns, y=corr.index,
            color_continuous_scale=CORR_CMAP, range_color=rng,
            aspect="auto", origin="upper"
        )
        text_vals = np.vectorize(lambda v: f"{v:.2f}")(corr.values)
        fig_corr.update_traces(
            text=text_vals,
            texttemplate="%{text}",
            hovertemplate="Row:%{y}<br>Col:%{x}<br>r:%{text}<extra></extra>",
            selector=dict(type="heatmap")
        )
        fig_corr.update_traces(textfont={"size": 11}, selector=dict(type="heatmap"))
        fig_corr.update_layout(
            margin=dict(l=20,r=20,t=10,b=20),
            coloraxis_colorbar=dict(title="r")
        )
        st.plotly_chart(fig_corr, use_container_width=True, key="corr_heatmap_all")
    else:
        st.info("상관관계를 계산할 월간 수익률 데이터가 없습니다.")

    # ---------- 19. Portfolio Return Decomposition ----------
    st.header("19. Portfolio Return Decomposition")
    if _mret_assets is not None and not _mret_assets.empty and st.session_state.portfolios:
        rows = []
        asset_list = list(_mret_assets.columns)
        contrib_cols = {}

        for pname, pstruct in st.session_state.portfolios.items():
            W = pd.Series({a:0.0 for a in asset_list}, dtype=float)
            total = 0.0
            for t, meta in pstruct.items():
                label = f"{t}{'(H)' if (st.session_state.base_currency!='USD' and not bool(meta.get('fx',False))) else ''}"
                if label in W.index:
                    w = float(meta.get("w", 0.0));
                    if w>0:
                        W[label] += w; total += w
            if total>0: W = W / total
            else: continue

            m = _mret_assets.reindex(columns=W.index).dropna(how="any")
            m = m.loc[m.index.intersection(_mret_ports.get(pname, pd.Series(dtype=float)).index)]
            if m.empty:
                continue

            V = float(st.session_state.initial_capital)
            asset_acc = pd.Series(0.0, index=W.index, dtype=float)
            for dt, row in m.iterrows():
                gains = V * (W * row)
                asset_acc = asset_acc.add(gains, fill_value=0.0)
                V = V * (1.0 + float((W * row).sum()))

            for a in W.index:
                contrib_cols.setdefault(a, {})[pname] = asset_acc.get(a, 0.0)

        if contrib_cols:
            df = pd.DataFrame(contrib_cols).T
            disp = df.applymap(lambda x: f"{sym}{x:,.0f}")
            disp.insert(0, "Ticker", disp.index)
            disp.insert(1, "Name", disp.index)
            st.dataframe(disp, use_container_width=True)
        else:
            st.info("수익 분해를 표시할 데이터가 없습니다.")
    else:
        st.info("수익 분해를 계산할 월간 데이터가 없습니다.")

    # ---------- 20. Portfolio Risk Decomposition ----------
    st.header("20. Portfolio Risk Decomposition")
    if _mret_assets is not None and not _mret_assets.empty and st.session_state.portfolios:
        m = _mret_assets.dropna(how="any")
        if not m.empty:
            Sigma = m.cov()
            out_frames = []
            for pname, pstruct in st.session_state.portfolios.items():
                W = pd.Series(0.0, index=m.columns, dtype=float)
                total = 0.0
                for t, meta in pstruct.items():
                    label = f"{t}{'(H)' if (st.session_state.base_currency!='USD' and not bool(meta.get('fx',False))) else ''}"
                    if label in W.index:
                        w = float(meta.get("w", 0.0))
                        if w>0:
                            W[label] += w; total += w
                if total<=0:
                    continue
                W = W/total
                port_var = float(W.T @ Sigma.values @ W.values)
                if port_var<=0 or not np.isfinite(port_var):
                    continue
                marginal = Sigma.values @ W.values
                contrib = W.values * marginal / port_var
                dfc = pd.DataFrame({pname: contrib*100.0}, index=m.columns)
                out_frames.append(dfc)
            if out_frames:
                RISK = pd.concat(out_frames, axis=1)
                disp = RISK.applymap(lambda x: f"{x:.2f}%")
                disp.insert(0, "Ticker", disp.index)
                disp.insert(1, "Name", disp.index)
                st.dataframe(disp, use_container_width=True)
            else:
                st.info("위험 분해를 표시할 데이터가 없습니다.")
        else:
            st.info("월간 공분산을 계산할 데이터가 없습니다.")
    else:
        st.info("위험 분해를 계산할 월간 데이터가 없습니다.")

    # ---------- 21. Annual Asset Returns ----------
    st.header("21. Annual Asset Returns")
    if _yret_assets is not None and not _yret_assets.empty:
        plot_df = _yret_assets.reset_index()
        plot_df["Year"] = plot_df["index"].dt.year
        plot_df = plot_df.drop(columns=["index"])
        plot_long = plot_df.melt(id_vars=["Year"], var_name="Asset", value_name="Return").dropna()
        if not plot_long.empty:
            fig_ay = px.bar(plot_long, x="Year", y="Return", color="Asset", barmode="group",
                            text=plot_long["Return"].map(lambda x: f"{x:.1%}"))
            fig_ay.update_traces(textposition="outside", cliponaxis=False)
            fig_ay.update_layout(height=460, margin=dict(l=20,r=20,t=30,b=80), yaxis_tickformat=".0%",
                                 legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5))
            st.plotly_chart(fig_ay, use_container_width=True, config={"displaylogo": False}, key="annual_assets_group")
        else:
            st.info("연간 수익률을 그릴 데이터가 없습니다.")
    else:
        st.info("자산의 연간 수익률 데이터가 없습니다.")

    # ---------- 22. Rolling Returns ----------
    st.header("22. Rolling Returns")
    if _mret_ports:
        rr_tbl = _rolling_table_for_portfolios(_mret_ports, years_list=(1,3,5,7))
        if not rr_tbl.empty:
            st.dataframe(rr_tbl, use_container_width=True)
        else:
            st.info("롤링 수익률 요약을 계산할 데이터가 부족합니다.")
        box_year = st.selectbox("Box Plot Period (years)", options=[1,3,5,7], index=1, key="rr_box_years")
        df_long = _rolling_ann_long_df(_mret_ports, years=int(box_year))
        if not df_long.empty:
            fig_box = px.box(df_long, x="Portfolio", y="AnnReturn", points=False)
            fig_box.update_layout(height=420, margin=dict(l=20,r=20,t=30,b=60),
                                  yaxis_tickformat=".0%",
                                  title=f"Rolling Annualized Returns — {int(box_year)} Year{'s' if int(box_year)>1 else ''}")
            st.plotly_chart(fig_box, use_container_width=True, key="rr_boxplot")
        else:
            st.info("박스플롯을 그릴 롤링 시계열이 없습니다.")
    else:
        st.info("포트폴리오 월간 수익률 데이터가 없습니다.")

    # ---------- 23. Annualized Rolling Return ----------
    st.header("23. Annualized Rolling Return")
    roll_years = st.selectbox("Window (years)", options=[1,3,5,7], index=1, key="rr_line_years")
    if _mret_ports or (_mret_bench is not None and not _mret_bench.empty):
        fig_roll = go.Figure()

        # ▶ 포트폴리오
        for pname, m in _mret_ports.items():
            ann = _rolling_ann_from_monthly(m, int(roll_years))
            if not ann.empty:
                fig_roll.add_trace(go.Scatter(x=ann.index, y=ann.values, mode="lines", name=pname))

        # ▶ 벤치마크
        if _mret_bench is not None and not _mret_bench.empty:
            annb = _rolling_ann_from_monthly(_mret_bench, int(roll_years))
            if not annb.empty:
                fig_roll.add_trace(go.Scatter(x=annb.index, y=annb.values, mode="lines", name="Benchmark"))

        fig_roll.update_layout(
            height=460, margin=dict(l=20,r=20,t=30,b=80),
            yaxis_tickformat=".0%",
            hovermode="x unified",
            title=f"Annualized Rolling Return — {int(roll_years)} Year{'s' if int(roll_years)>1 else ''}",
            legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5)
        )
        st.plotly_chart(fig_roll, use_container_width=True, key="rr_line")
    else:
        st.info("연환산 롤링 수익률을 계산할 시계열이 없습니다.")

# ---------- 내보내기 ----------
st.header("내보내기 (요약/값 시트)")
res=st.session_state.get("run_results")
if res:
    try:
        def _sig(res):
            names=sorted(list(res["values_dict"].keys()))
            vals_sig=tuple((n, int(len(res["values_dict"][n])), float(res["values_dict"][n].iloc[0]),
                            float(res["values_dict"][n].iloc[-1])) for n in names)
            bench_sig=None
            if res["bench_values"] is not None:
                bench_sig=(int(len(res["bench_values"])), float(res["bench_values"].iloc[0]), float(res["bench_values"].iloc[-1]))
            return (VERSION, str(res["global_start"]), str(res["global_end"]), res["base_currency"], tuple(names), vals_sig, bench_sig)
        current_sig=_sig(res)
        if st.session_state.__export_sig!=current_sig or st.session_state.__export_bytes is None:
            output=io.BytesIO()
            with pd.ExcelWriter(output, engine="openpyxl") as wr:
                weights_export={pf:{t:d.get("w",0.0) for t,d in st.session_state.portfolios[pf].items()}
                                for pf in st.session_state.portfolios.keys()}
                wdf=pd.DataFrame.from_dict(weights_export, orient="index").fillna(0.0); wdf.index.name="Portfolio"
                wdf.to_excel(wr, sheet_name="Weights(%)")
                gs,ge=res["global_start"],res["global_end"]
                plist=[]
                if res.get("bench_values") is not None: plist.append(res["bench_values"].loc[gs:ge].rename("Benchmark"))
                for name,s in res["values_dict"].items(): plist.append(s.loc[gs:ge].rename(name))
                if plist: pd.concat(plist, axis=1).to_excel(wr, sheet_name="Portfolio_Values")
                df_sum=st.session_state.run_results["metrics_df"].copy(); df_sum.to_excel(wr, sheet_name="Summary")
            output.seek(0); st.session_state.__export_bytes=output.read(); st.session_state.__export_sig=current_sig
        st.download_button(f"Excel(.xlsx) 다운로드 (v{VERSION})", data=st.session_state.__export_bytes,
                           file_name=f"backtest_export_v{VERSION}.xlsx",
                           mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
                           use_container_width=True)
    except Exception as e:
        st.error(f"Excel 생성 오류: {e}")
else:
    st.info("백테스트를 먼저 실행하면 내보내기 버튼이 활성화됩니다.")


Overwriting app.py


In [ ]:
# 4. Cloudflared로 외부 URL 열기 + Streamlit 실행
# v4.1.0 실행: Cloudflared로 외부 URL 노출 + Streamlit 기동
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

# 백그라운드로 앱 실행(이전 프로세스 종료 후 재기동)
!pkill -f "streamlit run app.py" >/dev/null 2>&1 || true
!streamlit run app.py &>/dev/null &

# 터널 시작 및 URL 추출
import re, subprocess, time

proc = subprocess.Popen(['./cloudflared','tunnel','--url','http://localhost:8501','--no-autoupdate'],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print("Cloudflare 터널 시작 중...")
url = ""
start = time.time()
while time.time() - start < 60:
    line = proc.stdout.readline()
    if not line:
        time.sleep(0.2)
        continue
    m = re.search(r"(https://[a-zA-Z0-9-]+\.trycloudflare\.com)", line)
    if m:
        url = m.group(1)
        break

print("="*70)
if url:
    print("🎉 접속 링크:", url)
else:
    print("터널 URL을 찾지 못했습니다. 위 로그를 확인하세요.")
print("="*70)

# (선택) 코랩 프록시 링크도 함께 출력
try:
    from google.colab import output
    proxy_url = output.eval_js('google.colab.kernel.proxyPort(8501, false)')
    print("🔗 Colab Proxy URL:", proxy_url)
except Exception:
    pass


cloudflared: Text file busy
^C
Cloudflare 터널 시작 중...
🎉 접속 링크: https://customers-gods-mind-loops.trycloudflare.com
🔗 Colab Proxy URL: https://8501-m-s-1h44z13g1nw0j-b.us-west1-2.prod.colab.dev
